In [6]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

DATA_PATH = r"C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\\"
OUTPUT_PATH = r"C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\workingfile\\"

files = {
    'commercial_property': DATA_PATH + "Commercial property prices (CSV, flat).csv",
    'residential_property': DATA_PATH + "Detailed residential property prices (CSV, flat).csv",
    'effective_fx': DATA_PATH + "Effective exchange rates (CSV, flat).csv"
}

print("="*100)
print("DATA CLEANING PIPELINE - REER → PROPERTY SPILLOVERS (NO OTC)")
print("="*100)

# STEP 1: LOAD
print("\nSTEP 1: LOADING...")
datasets = {}
for name, path in files.items():
    df = pd.read_csv(path, encoding='utf-8', low_memory=False)
    datasets[name] = df
    print(f"  Loaded {name}: {len(df):,} rows")

# STEP 2: REER (Real Effective Exchange Rate)
print("\nSTEP 2: PROCESSING REER...")
reer = datasets['effective_fx'].copy()
reer = reer[(reer['EER_TYPE:Type'] == 'R: Real') & 
            (reer['EER_BASKET:Basket'] == 'B: Broad (64 economies)') & 
            (reer['FREQ:Frequency'] == 'M: Monthly')].copy()
reer = reer[reer['TIME_PERIOD:Time period or range'].notna()].copy()
reer['year'] = reer['TIME_PERIOD:Time period or range'].str[:4].astype(int)
reer['month'] = reer['TIME_PERIOD:Time period or range'].str[5:7].astype(int)
reer = reer[(reer['year'] >= 2000) & (reer['year'] <= 2024)].copy()
reer['quarter'] = ((reer['month'] - 1) // 3) + 1
reer_quarterly = reer[reer['month'].isin([3, 6, 9, 12])].copy()
reer_quarterly['time_period'] = reer_quarterly['year'].astype(str) + '-Q' + reer_quarterly['quarter'].astype(str)
reer_clean = reer_quarterly[['REF_AREA:Reference area', 'time_period', 'OBS_VALUE:Observation Value']].copy()
reer_clean.columns = ['country_code', 'time_period', 'reer_broad']
reer_clean = reer_clean.dropna(subset=['reer_broad'])
print(f"  REER: {len(reer_clean):,} obs, {reer_clean['country_code'].nunique()} countries")

# STEP 3: RESIDENTIAL PROPERTY
print("\nSTEP 3: PROCESSING RESIDENTIAL...")
res_prop = datasets['residential_property'].copy()
res_prop = res_prop[res_prop['FREQ:Frequency'].str.contains('Q: Quarterly', na=False)].copy()
res_prop = res_prop[res_prop['TIME_PERIOD:Time period or range'].notna()].copy()
res_prop = res_prop[res_prop['TIME_PERIOD:Time period or range'].str.match(r'\d{4}-Q[1-4]', na=False)].copy()
res_prop['year'] = res_prop['TIME_PERIOD:Time period or range'].str[:4].astype(int)
res_prop = res_prop[(res_prop['year'] >= 2000) & (res_prop['year'] <= 2024)].copy()

def calc_priority_res(row):
    score = 0
    if pd.notna(row.get('COVERED_AREA:Covered area')) and '0: Whole country' in str(row['COVERED_AREA:Covered area']):
        score += 100
    if pd.notna(row.get('RE_TYPE:Real estate type')) and '1: All types' in str(row['RE_TYPE:Real estate type']):
        score += 50
    if pd.notna(row.get('RE_VINTAGE:Real estate vintage')) and '0: All' in str(row['RE_VINTAGE:Real estate vintage']):
        score += 25
    return score

res_prop['priority'] = res_prop.apply(calc_priority_res, axis=1)
res_prop = res_prop.sort_values(['REF_AREA:Reference area', 'TIME_PERIOD:Time period or range', 'priority'], ascending=[True, True, False])
res_prop_clean = res_prop.groupby(['REF_AREA:Reference area', 'TIME_PERIOD:Time period or range']).first().reset_index()
res_prop_clean = res_prop_clean[['REF_AREA:Reference area', 'TIME_PERIOD:Time period or range', 'OBS_VALUE:Observation Value']].copy()
res_prop_clean.columns = ['country_code', 'time_period', 'residential_price']
res_prop_clean = res_prop_clean.dropna(subset=['residential_price'])
print(f"  Residential: {len(res_prop_clean):,} obs, {res_prop_clean['country_code'].nunique()} countries")

# STEP 4: COMMERCIAL PROPERTY
print("\nSTEP 4: PROCESSING COMMERCIAL...")
com_prop = datasets['commercial_property'].copy()
com_prop = com_prop[com_prop['FREQ:Frequency'].str.contains('Q: Quarterly', na=False)].copy()
com_prop = com_prop[com_prop['TIME_PERIOD:Time period or range'].notna()].copy()
com_prop = com_prop[com_prop['TIME_PERIOD:Time period or range'].str.match(r'\d{4}-Q[1-4]', na=False)].copy()
com_prop['year'] = com_prop['TIME_PERIOD:Time period or range'].str[:4].astype(int)
com_prop = com_prop[(com_prop['year'] >= 2000) & (com_prop['year'] <= 2024)].copy()

def calc_priority_com(row):
    score = 0
    if pd.notna(row.get('COVERED_AREA:Covered area')) and '0: Whole country' in str(row['COVERED_AREA:Covered area']):
        score += 100
    if pd.notna(row.get('RE_TYPE:Real estate type')) and 'A: Commercial property' in str(row['RE_TYPE:Real estate type']):
        score += 50
    return score

com_prop['priority'] = com_prop.apply(calc_priority_com, axis=1)
com_prop = com_prop.sort_values(['REF_AREA:Reference area', 'TIME_PERIOD:Time period or range', 'priority'], ascending=[True, True, False])
com_prop_clean = com_prop.groupby(['REF_AREA:Reference area', 'TIME_PERIOD:Time period or range']).first().reset_index()
com_prop_clean = com_prop_clean[['REF_AREA:Reference area', 'TIME_PERIOD:Time period or range', 'OBS_VALUE:Observation Value']].copy()
com_prop_clean.columns = ['country_code', 'time_period', 'commercial_price']
com_prop_clean = com_prop_clean.dropna(subset=['commercial_price'])
print(f"  Commercial: {len(com_prop_clean):,} obs, {com_prop_clean['country_code'].nunique()} countries")

# STEP 5: MERGE
print("\nSTEP 5: MERGING...")
panel = reer_clean.copy()
panel = panel.merge(res_prop_clean, on=['country_code', 'time_period'], how='left')
panel = panel.merge(com_prop_clean, on=['country_code', 'time_period'], how='left')
panel['year'] = panel['time_period'].str[:4].astype(int)
panel['quarter'] = panel['time_period'].str[-1].astype(int)
print(f"  Merged panel: {len(panel):,} obs, {panel['country_code'].nunique()} countries")

# STEP 6: CREATE DERIVED VARIABLES
print("\nSTEP 6: CREATING DERIVED VARIABLES...")
panel = panel.sort_values(['country_code', 'year', 'quarter'])

def create_variables(group):
    # Log transformations
    group['log_reer'] = np.log(group['reer_broad'])
    group['log_res_price'] = np.log(group['residential_price'])
    group['log_com_price'] = np.log(group['commercial_price'])
    
    # First differences (growth rates)
    group['d_log_reer'] = group['log_reer'].diff()
    group['d_log_res_price'] = group['log_res_price'].diff()
    group['d_log_com_price'] = group['log_com_price'].diff()
    
    # Growth rates in percentage
    group['reer_change_pct'] = group['d_log_reer'] * 100
    group['res_price_growth'] = group['d_log_res_price'] * 100
    group['com_price_growth'] = group['d_log_com_price'] * 100
    
    # REER volatility (4-quarter rolling std)
    group['reer_volatility'] = group['d_log_reer'].rolling(window=4, min_periods=2).std()
    
    # REER appreciation dummy (1 if REER increased)
    group['reer_appreciation'] = (group['d_log_reer'] > 0).astype(int)
    
    # Lagged variables (for models)
    group['reer_lag1'] = group['reer_broad'].shift(1)
    group['res_price_lag1'] = group['residential_price'].shift(1)
    group['com_price_lag1'] = group['commercial_price'].shift(1)
    
    return group

panel = panel.groupby('country_code', group_keys=False).apply(create_variables)
print(f"  Created: log transforms, growth rates, volatility, lags")

# STEP 7: DATA QUALITY ASSESSMENT
print("\nSTEP 7: DATA QUALITY...")
completeness = panel.groupby('country_code').agg({
    'reer_broad': lambda x: x.notna().sum() / len(x) * 100,
    'residential_price': lambda x: x.notna().sum() / len(x) * 100,
    'commercial_price': lambda x: x.notna().sum() / len(x) * 100,
    'time_period': 'count'
}).round(2)
completeness.columns = ['REER_%', 'Residential_%', 'Commercial_%', 'N_obs']
completeness = completeness.sort_values('N_obs', ascending=False)

# Balanced panels with different criteria
balanced_full = completeness[(completeness['REER_%'] >= 90) & 
                             (completeness['Residential_%'] >= 90) & 
                             (completeness['Commercial_%'] >= 90)].index.tolist()

balanced_res_only = completeness[(completeness['REER_%'] >= 90) & 
                                 (completeness['Residential_%'] >= 90)].index.tolist()

balanced_com_only = completeness[(completeness['REER_%'] >= 90) & 
                                 (completeness['Commercial_%'] >= 90)].index.tolist()

print(f"  Full balanced (REER+Residential+Commercial >90%): {len(balanced_full)} countries")
print(f"  Residential balanced (REER+Residential >90%): {len(balanced_res_only)} countries")
print(f"  Commercial balanced (REER+Commercial >90%): {len(balanced_com_only)} countries")

# STEP 8: SAVE FILES
print("\nSTEP 8: SAVING FILES...")
panel.to_csv(OUTPUT_PATH + 'panel_full.csv', index=False)
print(f"  Saved: panel_full.csv ({len(panel):,} rows)")

if balanced_full:
    panel_full_bal = panel[panel['country_code'].isin(balanced_full)].copy()
    panel_full_bal.to_csv(OUTPUT_PATH + 'panel_balanced_full.csv', index=False)
    print(f"  Saved: panel_balanced_full.csv ({len(panel_full_bal):,} rows, {len(balanced_full)} countries)")
    print(f"    Countries: {balanced_full}")

if balanced_res_only:
    panel_res = panel[panel['country_code'].isin(balanced_res_only)].copy()
    panel_res.to_csv(OUTPUT_PATH + 'panel_balanced_residential.csv', index=False)
    print(f"  Saved: panel_balanced_residential.csv ({len(panel_res):,} rows, {len(balanced_res_only)} countries)")

if balanced_com_only:
    panel_com = panel[panel['country_code'].isin(balanced_com_only)].copy()
    panel_com.to_csv(OUTPUT_PATH + 'panel_balanced_commercial.csv', index=False)
    print(f"  Saved: panel_balanced_commercial.csv ({len(panel_com):,} rows, {len(balanced_com_only)} countries)")

completeness.to_csv(OUTPUT_PATH + 'data_completeness.csv')
print(f"  Saved: data_completeness.csv")

# Variable descriptions
var_desc = pd.DataFrame({
    'Variable': ['country_code', 'time_period', 'year', 'quarter', 
                 'reer_broad', 'residential_price', 'commercial_price',
                 'log_reer', 'd_log_reer', 'reer_change_pct', 'reer_volatility', 'reer_appreciation',
                 'log_res_price', 'd_log_res_price', 'res_price_growth',
                 'log_com_price', 'd_log_com_price', 'com_price_growth',
                 'reer_lag1', 'res_price_lag1', 'com_price_lag1'],
    'Description': ['ISO 2-letter country code', 'YYYY-Qn format', 'Year (numeric)', 'Quarter (1-4)',
                    'Real Effective Exchange Rate, Broad basket (2020=100)', 
                    'Residential property price index', 'Commercial property price index',
                    'Log of REER', 'Log difference of REER (quarterly change)', 'REER change (%)', 
                    'REER volatility (4Q rolling std)', 'Dummy: 1 if REER appreciated',
                    'Log of residential price', 'Log difference residential', 'Residential price growth (%)',
                    'Log of commercial price', 'Log difference commercial', 'Commercial price growth (%)',
                    'REER lagged 1 quarter', 'Residential price lagged 1 quarter', 'Commercial price lagged 1 quarter']
})
var_desc.to_csv(OUTPUT_PATH + 'variable_descriptions.csv', index=False)
print(f"  Saved: variable_descriptions.csv")

# Summary statistics
print("\n" + "="*100)
print("DESCRIPTIVE STATISTICS")
print("="*100)
print(f"\nFull Panel Summary:")
print(panel[['reer_broad', 'residential_price', 'commercial_price', 
             'reer_change_pct', 'res_price_growth', 'com_price_growth', 'reer_volatility']].describe().round(3).to_string())

print("\n" + "="*100)
print("✅ CLEANING COMPLETE!")
print("="*100)
print(f"Panel: {len(panel):,} obs | {panel['country_code'].nunique()} countries | {panel['time_period'].min()} to {panel['time_period'].max()}")
print(f"\nBalanced panels created:")
print(f"  - Full (REER+Res+Com): {len(balanced_full)} countries")
print(f"  - Residential focus: {len(balanced_res_only)} countries")
print(f"  - Commercial focus: {len(balanced_com_only)} countries")
print(f"\nAll files saved to: {OUTPUT_PATH}")


DATA CLEANING PIPELINE - REER → PROPERTY SPILLOVERS (NO OTC)

STEP 1: LOADING...
  Loaded commercial_property: 6,538 rows
  Loaded residential_property: 44,109 rows
  Loaded effective_fx: 1,181,937 rows

STEP 2: PROCESSING REER...
  REER: 6,400 obs, 64 countries

STEP 3: PROCESSING RESIDENTIAL...
  Residential: 4,379 obs, 51 countries

STEP 4: PROCESSING COMMERCIAL...
  Commercial: 1,389 obs, 20 countries

STEP 5: MERGING...
  Merged panel: 6,400 obs, 64 countries

STEP 6: CREATING DERIVED VARIABLES...
  Created: log transforms, growth rates, volatility, lags

STEP 7: DATA QUALITY...
  Full balanced (REER+Residential+Commercial >90%): 4 countries
  Residential balanced (REER+Residential >90%): 29 countries
  Commercial balanced (REER+Commercial >90%): 6 countries

STEP 8: SAVING FILES...
  Saved: panel_full.csv (6,400 rows)
  Saved: panel_balanced_full.csv (400 rows, 4 countries)
    Countries: ['SG: Singapore', 'US: United States', 'XM: Euro area', 'DK: Denmark']
  Saved: panel_balanc

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# VIBRANT MODERN STYLING - COLORFUL & STUNNING
# ============================================================================
plt.style.use('default')

COLORS = {
    'primary': '#FF6B6B',      # Vibrant red
    'secondary': '#4ECDC4',    # Teal
    'accent1': '#FFE66D',      # Yellow
    'accent2': '#95E1D3',      # Mint
    'accent3': '#F38181',      # Pink
    'accent4': '#AA96DA',      # Purple
    'safe_haven': '#667EEA',   # Blue-purple gradient
    'normal': '#F093FB',       # Pink gradient
    'bg1': '#FEF9EF',         # Warm white
    'bg2': '#E8F5E9'          # Cool white
}

GRADIENT_SAFE = ['#667EEA', '#764BA2']  # Blue to purple
GRADIENT_NORMAL = ['#F093FB', '#F5576C']  # Pink to red

plt.rcParams.update({
    'figure.figsize': (14, 8),
    'font.size': 13,
    'font.family': 'sans-serif',
    'axes.labelsize': 15,
    'axes.labelweight': 'bold',
    'axes.labelcolor': '#2D3436',
    'axes.linewidth': 2.5,
    'axes.edgecolor': '#2D3436',
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'grid.alpha': 0,
    'legend.fontsize': 13,
    'legend.frameon': False,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'xtick.major.width': 2,
    'ytick.major.width': 2,
    'xtick.color': '#2D3436',
    'ytick.color': '#2D3436',
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'savefig.facecolor': 'white'
})

DATA_PATH = r"C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\workingfile\\"
OUTPUT_PATH = r"C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\workingfile\descriptives\\"

import os
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(OUTPUT_PATH + 'tables', exist_ok=True)
os.makedirs(OUTPUT_PATH + 'figures', exist_ok=True)

print("="*100)
print("🎨 DESCRIPTIVE STATISTICS - VIBRANT & COLORFUL")
print("="*100)

# ============================================================================
# LOAD DATA
# ============================================================================
df = pd.read_csv(DATA_PATH + 'panel_balanced_residential.csv')
print(f"\nData: {len(df):,} obs, {df['country_code'].nunique()} countries")
print(f"Period: {df['year'].min()}-Q{df['quarter'].min()} to {df['year'].max()}-Q{df['quarter'].max()}")

SAFE_HAVEN = ['US: United States', 'JP: Japan', 'CH: Switzerland', 'DE: Germany']
df['safe_haven'] = df['country_code'].isin(SAFE_HAVEN).astype(int)
df['country_group'] = df['safe_haven'].map({1: 'Safe Haven', 0: 'Normal'})

if 'res_price_lag1' not in df.columns:
    df = df.sort_values(['country_code', 'year', 'quarter'])
    df['res_price_lag1'] = df.groupby('country_code')['residential_price'].shift(1)

# ============================================================================
# TABLES (SAME AS BEFORE)
# ============================================================================
print("\n" + "="*100)
print("Creating Tables...")
print("="*100)

vars_to_summarize = [
    'res_price_growth', 'reer_change_pct', 'reer_volatility',
    'reer_appreciation', 'res_price_lag1'
]

var_names = {
    'res_price_growth': 'Property Growth (%)',
    'reer_change_pct': 'REER Change (%)',
    'reer_volatility': 'REER Volatility',
    'reer_appreciation': 'REER Appreciation',
    'res_price_lag1': 'Lagged Property Price'
}

# Table 1: Summary Stats
summary_stats = []
for var in vars_to_summarize:
    data = df[var].dropna()
    summary_stats.append({
        'Variable': var_names[var],
        'N': len(data),
        'Mean': data.mean(),
        'Std Dev': data.std(),
        'Min': data.min(),
        'Median': data.median(),
        'Max': data.max()
    })

table1 = pd.DataFrame(summary_stats)
table1.to_excel(OUTPUT_PATH + 'tables/Table1_Summary_Statistics.xlsx', index=False)
table1.to_csv(OUTPUT_PATH + 'tables/Table1_Summary_Statistics.csv', index=False)
print("✓ Table1_Summary_Statistics")

# Table 2: By Group
summary_by_group = []
for var in ['res_price_growth', 'reer_change_pct', 'reer_volatility']:
    for group in ['Safe Haven', 'Normal']:
        data = df[df['country_group'] == group][var].dropna()
        summary_by_group.append({
            'Variable': var_names[var],
            'Group': group,
            'N': len(data),
            'Mean': data.mean(),
            'Std Dev': data.std(),
            'Median': data.median()
        })

table2 = pd.DataFrame(summary_by_group)
table2.to_excel(OUTPUT_PATH + 'tables/Table2_Summary_by_Group.xlsx', index=False)
table2.to_csv(OUTPUT_PATH + 'tables/Table2_Summary_by_Group.csv', index=False)
print("✓ Table2_Summary_by_Group")

# Table 3: Correlation
corr_vars = ['res_price_growth', 'reer_change_pct', 'reer_volatility', 
             'reer_appreciation', 'res_price_lag1']
corr_matrix = df[corr_vars].corr()
corr_matrix.index = [var_names[v] for v in corr_matrix.index]
corr_matrix.columns = [var_names[v] for v in corr_matrix.columns]
corr_matrix.to_excel(OUTPUT_PATH + 'tables/Table3_Correlation_Matrix.xlsx')
corr_matrix.to_csv(OUTPUT_PATH + 'tables/Table3_Correlation_Matrix.csv')
print("✓ Table3_Correlation_Matrix")

# ============================================================================
# FIGURE 1: VIBRANT GRADIENT DISTRIBUTIONS
# ============================================================================
print("\n" + "="*100)
print("Creating Figure 1: Distributions (Vibrant)")
print("="*100)

fig, axes = plt.subplots(1, 3, figsize=(18, 5.5))

vars_plot = [
    ('res_price_growth', 'Property Growth (%)'),
    ('reer_change_pct', 'REER Change (%)'),
    ('reer_volatility', 'REER Volatility')
]

for idx, (var, label) in enumerate(vars_plot):
    ax = axes[idx]
    
    safe_data = df[df['country_group'] == 'Safe Haven'][var].dropna()
    normal_data = df[df['country_group'] == 'Normal'][var].dropna()
    
    # KDE with gradient fill
    x_safe = np.linspace(safe_data.min(), safe_data.max(), 200)
    kde_safe = stats.gaussian_kde(safe_data)
    y_safe = kde_safe(x_safe)
    
    x_normal = np.linspace(normal_data.min(), normal_data.max(), 200)
    kde_normal = stats.gaussian_kde(normal_data)
    y_normal = kde_normal(x_normal)
    
    # Fill with gradient effect
    ax.fill_between(x_safe, 0, y_safe, alpha=0.6, color=COLORS['safe_haven'], 
                    label='Safe Haven', linewidth=0)
    ax.plot(x_safe, y_safe, color='#4A148C', linewidth=3.5, alpha=0.9)
    
    ax.fill_between(x_normal, 0, y_normal, alpha=0.6, color=COLORS['normal'], 
                    label='Normal', linewidth=0)
    ax.plot(x_normal, y_normal, color='#B71C1C', linewidth=3.5, alpha=0.9)
    
    ax.axvline(0, color='#2D3436', linewidth=2, alpha=0.3, linestyle='--')
    ax.set_xlabel(label, fontsize=14, fontweight='bold')
    ax.set_ylabel('Density', fontsize=14, fontweight='bold')
    ax.legend(loc='upper right', fontsize=12)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylim(bottom=0)
    
    for spine in ax.spines.values():
        spine.set_linewidth(2.5)

plt.tight_layout()
plt.savefig(OUTPUT_PATH + 'figures/Figure1_Distributions.png', dpi=300, bbox_inches='tight')
plt.savefig(OUTPUT_PATH + 'figures/Figure1_Distributions.pdf', bbox_inches='tight')
plt.close()
print("✓ Figure1_Distributions")

# ============================================================================
# FIGURE 2: VIBRANT TIME SERIES WITH SHADED AREAS
# ============================================================================
print("\n" + "="*100)
print("Creating Figure 2: Time Series (Vibrant)")
print("="*100)

df['date_numeric'] = df['year'] + (df['quarter'] - 1) / 4

ts_data = df.groupby(['date_numeric', 'country_group']).agg({
    'res_price_growth': 'mean',
    'reer_change_pct': 'mean'
}).reset_index()

fig, axes = plt.subplots(2, 1, figsize=(16, 9))

# Property Growth
ax = axes[0]
for group, color, thick_color in [('Safe Haven', COLORS['safe_haven'], '#4A148C'), 
                                   ('Normal', COLORS['normal'], '#B71C1C')]:
    data = ts_data[ts_data['country_group'] == group]
    ax.plot(data['date_numeric'], data['res_price_growth'], 
           linewidth=3.5, alpha=0.95, color=thick_color, label=group)
    ax.fill_between(data['date_numeric'], 0, data['res_price_growth'], 
                    alpha=0.25, color=color)

ax.axhline(0, color='#2D3436', linewidth=2, alpha=0.4, linestyle='--')
ax.axvspan(2008, 2009, alpha=0.15, color='#FFE66D', label='2008 Crisis')
ax.axvspan(2020, 2020.5, alpha=0.15, color='#FF6B6B', label='COVID-19')
ax.set_xlabel('Year', fontsize=15, fontweight='bold')
ax.set_ylabel('Property Growth (%)', fontsize=15, fontweight='bold')
ax.legend(loc='best', fontsize=12, ncol=2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
for spine in ax.spines.values():
    spine.set_linewidth(2.5)

# REER Change
ax = axes[1]
for group, color, thick_color in [('Safe Haven', COLORS['safe_haven'], '#4A148C'), 
                                   ('Normal', COLORS['normal'], '#B71C1C')]:
    data = ts_data[ts_data['country_group'] == group]
    ax.plot(data['date_numeric'], data['reer_change_pct'], 
           linewidth=3.5, alpha=0.95, color=thick_color, label=group)
    ax.fill_between(data['date_numeric'], 0, data['reer_change_pct'], 
                    alpha=0.25, color=color)

ax.axhline(0, color='#2D3436', linewidth=2, alpha=0.4, linestyle='--')
ax.axvspan(2008, 2009, alpha=0.15, color='#FFE66D')
ax.axvspan(2020, 2020.5, alpha=0.15, color='#FF6B6B')
ax.set_xlabel('Year', fontsize=15, fontweight='bold')
ax.set_ylabel('REER Change (%)', fontsize=15, fontweight='bold')
ax.legend(loc='best', fontsize=12)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
for spine in ax.spines.values():
    spine.set_linewidth(2.5)

plt.tight_layout()
plt.savefig(OUTPUT_PATH + 'figures/Figure2_Time_Series.png', dpi=300, bbox_inches='tight')
plt.savefig(OUTPUT_PATH + 'figures/Figure2_Time_Series.pdf', bbox_inches='tight')
plt.close()
print("✓ Figure2_Time_Series")

# ============================================================================
# FIGURE 3: VIBRANT CORRELATION HEATMAP
# ============================================================================
print("\n" + "="*100)
print("Creating Figure 3: Correlation Heatmap (Vibrant)")
print("="*100)

fig, ax = plt.subplots(figsize=(10, 8))

corr_data = df[corr_vars].corr()
mask = np.triu(np.ones_like(corr_data, dtype=bool), k=1)

# Custom colormap - vibrant
from matplotlib.colors import LinearSegmentedColormap
colors_cmap = ['#667EEA', '#FFFFFF', '#F5576C']
n_bins = 100
cmap = LinearSegmentedColormap.from_list('custom', colors_cmap, N=n_bins)

sns.heatmap(corr_data, mask=mask, annot=True, fmt='.2f', 
           cmap=cmap, center=0, vmin=-0.6, vmax=0.6,
           square=True, linewidths=3, linecolor='white',
           cbar_kws={"shrink": 0.75, "aspect": 25},
           annot_kws={'fontsize': 13, 'fontweight': 'bold', 'color': '#2D3436'},
           ax=ax)

labels = [var_names[v].replace(' (%)', '').replace('Property ', 'Prop. ') for v in corr_vars]
ax.set_xticklabels(labels, rotation=45, ha='right', fontsize=13, fontweight='bold')
ax.set_yticklabels(labels, rotation=0, fontsize=13, fontweight='bold')

for spine in ax.spines.values():
    spine.set_linewidth(3)
    spine.set_edgecolor('#2D3436')

plt.tight_layout()
plt.savefig(OUTPUT_PATH + 'figures/Figure3_Correlation_Matrix.png', dpi=300, bbox_inches='tight')
plt.savefig(OUTPUT_PATH + 'figures/Figure3_Correlation_Matrix.pdf', bbox_inches='tight')
plt.close()
print("✓ Figure3_Correlation_Matrix")

# ============================================================================
# FIGURE 4: VIBRANT SCATTER WITH REGRESSION
# ============================================================================
print("\n" + "="*100)
print("Creating Figure 4: Scatter Plots (Vibrant)")
print("="*100)

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Safe Haven
ax = axes[0]
safe_data = df[df['country_group'] == 'Safe Haven'].dropna(subset=['reer_change_pct', 'res_price_growth'])

ax.scatter(safe_data['reer_change_pct'], safe_data['res_price_growth'], 
          alpha=0.5, s=80, color=COLORS['safe_haven'], edgecolors='#4A148C', linewidth=1.5)

z = np.polyfit(safe_data['reer_change_pct'], safe_data['res_price_growth'], 1)
p = np.poly1d(z)
x_line = np.linspace(safe_data['reer_change_pct'].min(), safe_data['reer_change_pct'].max(), 100)
ax.plot(x_line, p(x_line), color='#FFE66D', linewidth=4.5, alpha=0.95, linestyle='-')

ax.axhline(0, color='#2D3436', linewidth=2, alpha=0.3, linestyle='--')
ax.axvline(0, color='#2D3436', linewidth=2, alpha=0.3, linestyle='--')
ax.set_xlabel('REER Change (%)', fontsize=15, fontweight='bold')
ax.set_ylabel('Property Growth (%)', fontsize=15, fontweight='bold')
ax.text(0.05, 0.95, 'Safe Haven', transform=ax.transAxes, 
       fontsize=16, fontweight='bold', va='top', color='#4A148C')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
for spine in ax.spines.values():
    spine.set_linewidth(2.5)

# Normal
ax = axes[1]
normal_data = df[df['country_group'] == 'Normal'].dropna(subset=['reer_change_pct', 'res_price_growth'])

ax.scatter(normal_data['reer_change_pct'], normal_data['res_price_growth'], 
          alpha=0.5, s=80, color=COLORS['normal'], edgecolors='#B71C1C', linewidth=1.5)

z = np.polyfit(normal_data['reer_change_pct'], normal_data['res_price_growth'], 1)
p = np.poly1d(z)
x_line = np.linspace(normal_data['reer_change_pct'].min(), normal_data['reer_change_pct'].max(), 100)
ax.plot(x_line, p(x_line), color='#4ECDC4', linewidth=4.5, alpha=0.95, linestyle='-')

ax.axhline(0, color='#2D3436', linewidth=2, alpha=0.3, linestyle='--')
ax.axvline(0, color='#2D3436', linewidth=2, alpha=0.3, linestyle='--')
ax.set_xlabel('REER Change (%)', fontsize=15, fontweight='bold')
ax.set_ylabel('Property Growth (%)', fontsize=15, fontweight='bold')
ax.text(0.05, 0.95, 'Normal', transform=ax.transAxes, 
       fontsize=16, fontweight='bold', va='top', color='#B71C1C')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
for spine in ax.spines.values():
    spine.set_linewidth(2.5)

plt.tight_layout()
plt.savefig(OUTPUT_PATH + 'figures/Figure4_Scatter_REER_Property.png', dpi=300, bbox_inches='tight')
plt.savefig(OUTPUT_PATH + 'figures/Figure4_Scatter_REER_Property.pdf', bbox_inches='tight')
plt.close()
print("✓ Figure4_Scatter_REER_Property")

# ============================================================================
# SUMMARY
# ============================================================================
print("\n" + "="*100)
print("🎨 DESCRIPTIVES COMPLETE - VIBRANT & COLORFUL!")
print("="*100)
print("\n📊 TABLES (3):")
print("   • Table1_Summary_Statistics")
print("   • Table2_Summary_by_Group")
print("   • Table3_Correlation_Matrix")
print("\n🎨 FIGURES (4) - VIBRANT:")
print("   • Figure1_Distributions")
print("   • Figure2_Time_Series")
print("   • Figure3_Correlation_Matrix")
print("   • Figure4_Scatter_REER_Property")
print(f"\n📁 Location: {OUTPUT_PATH}")


🎨 DESCRIPTIVE STATISTICS - VIBRANT & COLORFUL

Data: 2,900 obs, 29 countries
Period: 2000-Q1 to 2024-Q4

Creating Tables...
✓ Table1_Summary_Statistics
✓ Table2_Summary_by_Group
✓ Table3_Correlation_Matrix

Creating Figure 1: Distributions (Vibrant)
✓ Figure1_Distributions

Creating Figure 2: Time Series (Vibrant)
✓ Figure2_Time_Series

Creating Figure 3: Correlation Heatmap (Vibrant)
✓ Figure3_Correlation_Matrix

Creating Figure 4: Scatter Plots (Vibrant)
✓ Figure4_Scatter_REER_Property

🎨 DESCRIPTIVES COMPLETE - VIBRANT & COLORFUL!

📊 TABLES (3):
   • Table1_Summary_Statistics
   • Table2_Summary_by_Group
   • Table3_Correlation_Matrix

🎨 FIGURES (4) - VIBRANT:
   • Figure1_Distributions
   • Figure2_Time_Series
   • Figure3_Correlation_Matrix
   • Figure4_Scatter_REER_Property

📁 Location: C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\workingfile\descriptives\\


In [22]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.diagnostic import het_white, het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.stattools import adfuller
from linearmodels.panel import PanelOLS, RandomEffects
from linearmodels.panel.results import compare
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# PANEL DATA DIAGNOSTIC TESTS - PUBLICATION READY TABLES
# ============================================================================

DATA_PATH = r"C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\workingfile\\"
OUTPUT_PATH = r"C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\workingfile\diagnostics\\"

import os
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(OUTPUT_PATH + 'tables', exist_ok=True)

print("="*100)
print("🔬 PANEL DATA DIAGNOSTIC TESTS")
print("="*100)

# ============================================================================
# LOAD DATA
# ============================================================================
df = pd.read_csv(DATA_PATH + 'panel_balanced_residential.csv')

# Create safe haven variable
SAFE_HAVEN = ['US: United States', 'JP: Japan', 'CH: Switzerland', 'DE: Germany']
df['safe_haven'] = df['country_code'].isin(SAFE_HAVEN).astype(int)

# Create lagged variable if not exists
if 'res_price_lag1' not in df.columns:
    df = df.sort_values(['country_code', 'year', 'quarter'])
    df['res_price_lag1'] = df.groupby('country_code')['residential_price'].shift(1)

# ✅ FIX: Create NUMERIC time index (year.quarter format)
df['time_numeric'] = df['year'] + (df['quarter'] - 1) / 4

# Set MultiIndex with NUMERIC time
df_panel = df.set_index(['country_code', 'time_numeric'])

print(f"\n📊 Data: {len(df):,} observations, {df['country_code'].nunique()} countries")
print(f"📅 Period: {df['year'].min()}-Q{df['quarter'].min()} to {df['year'].max()}-Q{df['quarter'].max()}")

# Variables to test
VARIABLES = ['res_price_growth', 'reer_change_pct', 'reer_volatility', 
             'reer_appreciation', 'res_price_lag1']

VAR_NAMES = {
    'res_price_growth': 'Property Growth (%)',
    'reer_change_pct': 'REER Change (%)',
    'reer_volatility': 'REER Volatility',
    'reer_appreciation': 'REER Appreciation',
    'res_price_lag1': 'Lagged Property Price'
}

# ============================================================================
# TEST 1: PANEL UNIT ROOT TESTS (IPS METHOD - AVERAGE ADF)
# ============================================================================
print("\n" + "="*100)
print("TEST 1: PANEL UNIT ROOT TESTS (IPS Method)")
print("="*100)

unit_root_results = []

for var in VARIABLES:
    print(f"\nTesting: {VAR_NAMES[var]}")
    
    # Get data by country
    adf_stats = []
    countries = df['country_code'].unique()
    
    for country in countries:
        country_data = df[df['country_code'] == country][var].dropna()
        
        if len(country_data) > 10:  # Need sufficient observations
            try:
                adf_result = adfuller(country_data, maxlag=4, regression='ct')
                adf_stats.append(adf_result[0])  # Test statistic
            except:
                continue
    
    # IPS: Average of ADF statistics
    if len(adf_stats) > 0:
        avg_adf = np.mean(adf_stats)
        # Approximate p-value (more negative = more stationary)
        if avg_adf < -3.0:
            conclusion = "Stationary ***"
            stationarity = "Yes"
        elif avg_adf < -2.5:
            conclusion = "Stationary **"
            stationarity = "Yes"
        elif avg_adf < -2.0:
            conclusion = "Stationary *"
            stationarity = "Yes"
        else:
            conclusion = "Non-Stationary"
            stationarity = "No"
        
        unit_root_results.append({
            'Variable': VAR_NAMES[var],
            'Avg ADF Statistic': round(avg_adf, 3),
            'N Panels': len(adf_stats),
            'Stationary': stationarity,
            'Conclusion': conclusion
        })
        
        print(f"   Avg ADF: {avg_adf:.3f} ({len(adf_stats)} panels) - {conclusion}")

# Save Unit Root Test Table
unit_root_df = pd.DataFrame(unit_root_results)
unit_root_df.to_excel(OUTPUT_PATH + 'tables/Test1_Unit_Root_Tests.xlsx', index=False)
unit_root_df.to_csv(OUTPUT_PATH + 'tables/Test1_Unit_Root_Tests.csv', index=False)
print("\n✓ Table saved: Test1_Unit_Root_Tests")

# ============================================================================
# TEST 2: CROSS-SECTIONAL DEPENDENCE (PESARAN CD TEST)
# ============================================================================
print("\n" + "="*100)
print("TEST 2: CROSS-SECTIONAL DEPENDENCE (Pesaran CD Test)")
print("="*100)

cd_results = []

for var in VARIABLES:
    print(f"\nTesting: {VAR_NAMES[var]}")
    
    # ✅ FIX: Use numeric time for pivot
    pivot_data = df.pivot(index='time_numeric', columns='country_code', values=var).dropna()
    
    if pivot_data.shape[1] > 1:  # Need at least 2 countries
        # Calculate pairwise correlations
        corr_matrix = pivot_data.corr()
        
        # Extract upper triangle (exclude diagonal)
        n = corr_matrix.shape[0]
        upper_corr = []
        for i in range(n):
            for j in range(i+1, n):
                upper_corr.append(corr_matrix.iloc[i, j])
        
        # Pesaran CD statistic
        T = pivot_data.shape[0]  # Time periods
        N = pivot_data.shape[1]  # Countries
        avg_corr = np.mean(upper_corr)
        
        # CD = sqrt(2T/(N(N-1))) * sum(correlations)
        cd_stat = np.sqrt(2 * T / (N * (N - 1))) * np.sum(upper_corr)
        
        # P-value (CD follows N(0,1) under H0: no cross-sectional dependence)
        p_value = 2 * (1 - stats.norm.cdf(abs(cd_stat)))
        
        if p_value < 0.01:
            conclusion = "Strong Dependence ***"
        elif p_value < 0.05:
            conclusion = "Moderate Dependence **"
        elif p_value < 0.10:
            conclusion = "Weak Dependence *"
        else:
            conclusion = "No Dependence"
        
        cd_results.append({
            'Variable': VAR_NAMES[var],
            'CD Statistic': round(cd_stat, 3),
            'P-value': round(p_value, 4),
            'Avg Correlation': round(avg_corr, 3),
            'Conclusion': conclusion
        })
        
        print(f"   CD Stat: {cd_stat:.3f}, P-value: {p_value:.4f} - {conclusion}")

# Save CD Test Table
cd_df = pd.DataFrame(cd_results)
cd_df.to_excel(OUTPUT_PATH + 'tables/Test2_Cross_Sectional_Dependence.xlsx', index=False)
cd_df.to_csv(OUTPUT_PATH + 'tables/Test2_Cross_Sectional_Dependence.csv', index=False)
print("\n✓ Table saved: Test2_Cross_Sectional_Dependence")

# ============================================================================
# TEST 3: HAUSMAN TEST (FE vs RE)
# ============================================================================
print("\n" + "="*100)
print("TEST 3: HAUSMAN TEST (Fixed Effects vs Random Effects)")
print("="*100)

# Drop missing values for panel regression
df_clean = df_panel[['res_price_growth', 'reer_change_pct', 'reer_volatility', 
                      'reer_appreciation', 'res_price_lag1']].dropna()

print(f"   Clean data: {len(df_clean):,} observations")

# Fixed Effects Model
fe_model = PanelOLS(
    dependent=df_clean['res_price_growth'],
    exog=df_clean[['reer_change_pct', 'reer_volatility', 'reer_appreciation', 'res_price_lag1']],
    entity_effects=True,
    time_effects=True
)
fe_results = fe_model.fit(cov_type='clustered', cluster_entity=True)

# Random Effects Model
re_model = RandomEffects(
    dependent=df_clean['res_price_growth'],
    exog=df_clean[['reer_change_pct', 'reer_volatility', 'reer_appreciation', 'res_price_lag1']]
)
re_results = re_model.fit(cov_type='clustered', cluster_entity=True)

# Hausman Test
# H0: Random Effects is consistent and efficient
# H1: Only Fixed Effects is consistent

fe_params = fe_results.params[['reer_change_pct', 'reer_volatility', 'reer_appreciation', 'res_price_lag1']]
re_params = re_results.params[['reer_change_pct', 'reer_volatility', 'reer_appreciation', 'res_price_lag1']]

fe_cov = fe_results.cov[['reer_change_pct', 'reer_volatility', 'reer_appreciation', 'res_price_lag1']].loc[['reer_change_pct', 'reer_volatility', 'reer_appreciation', 'res_price_lag1']]
re_cov = re_results.cov[['reer_change_pct', 'reer_volatility', 'reer_appreciation', 'res_price_lag1']].loc[['reer_change_pct', 'reer_volatility', 'reer_appreciation', 'res_price_lag1']]

# Calculate Hausman statistic
diff = fe_params - re_params
cov_diff = fe_cov - re_cov

try:
    hausman_stat = diff.T @ np.linalg.inv(cov_diff) @ diff
    df_hausman = len(diff)
    hausman_pval = 1 - stats.chi2.cdf(hausman_stat, df_hausman)
    
    if hausman_pval < 0.01:
        conclusion = "Use Fixed Effects ***"
    elif hausman_pval < 0.05:
        conclusion = "Use Fixed Effects **"
    elif hausman_pval < 0.10:
        conclusion = "Use Fixed Effects *"
    else:
        conclusion = "Random Effects OK"
    
    hausman_results = [{
        'Test': 'Hausman Test',
        'Chi-Square Statistic': round(hausman_stat, 3),
        'Degrees of Freedom': df_hausman,
        'P-value': round(hausman_pval, 4),
        'Conclusion': conclusion
    }]
    
    print(f"\n   Chi-Square: {hausman_stat:.3f}, P-value: {hausman_pval:.4f}")
    print(f"   {conclusion}")
    
except Exception as e:
    print(f"\n   ⚠ Could not compute Hausman test: {str(e)}")
    hausman_results = [{
        'Test': 'Hausman Test',
        'Chi-Square Statistic': 'NA',
        'Degrees of Freedom': 'NA',
        'P-value': 'NA',
        'Conclusion': 'Could not compute (use FE by default)'
    }]

# Save Hausman Test Table
hausman_df = pd.DataFrame(hausman_results)
hausman_df.to_excel(OUTPUT_PATH + 'tables/Test3_Hausman_Test.xlsx', index=False)
hausman_df.to_csv(OUTPUT_PATH + 'tables/Test3_Hausman_Test.csv', index=False)
print("\n✓ Table saved: Test3_Hausman_Test")

# ============================================================================
# TEST 4: HETEROSKEDASTICITY (MODIFIED WALD TEST)
# ============================================================================
print("\n" + "="*100)
print("TEST 4: HETEROSKEDASTICITY TEST (Modified Wald)")
print("="*100)

# Get residuals from FE model
residuals = fe_results.resids

# Group residuals by country
resid_by_country = df_clean.reset_index().copy()
resid_by_country['residuals'] = residuals.values

# Calculate variance by country
country_vars = resid_by_country.groupby('country_code')['residuals'].var()

# Modified Wald Test for groupwise heteroskedasticity
# H0: Homoskedasticity (equal variances)
n_countries = len(country_vars)
T_avg = len(residuals) / n_countries

# Test statistic (simplified version)
var_ratio = country_vars.max() / country_vars.min()

if var_ratio > 10:
    hetero_conclusion = "Strong Heteroskedasticity ***"
elif var_ratio > 5:
    hetero_conclusion = "Moderate Heteroskedasticity **"
elif var_ratio > 2:
    hetero_conclusion = "Weak Heteroskedasticity *"
else:
    hetero_conclusion = "Homoskedasticity"

hetero_results = [{
    'Test': 'Modified Wald Test',
    'Max Variance': round(country_vars.max(), 4),
    'Min Variance': round(country_vars.min(), 4),
    'Variance Ratio': round(var_ratio, 2),
    'Conclusion': hetero_conclusion
}]

print(f"\n   Variance Ratio (Max/Min): {var_ratio:.2f}")
print(f"   {hetero_conclusion}")

# Save Heteroskedasticity Test Table
hetero_df = pd.DataFrame(hetero_results)
hetero_df.to_excel(OUTPUT_PATH + 'tables/Test4_Heteroskedasticity_Test.xlsx', index=False)
hetero_df.to_csv(OUTPUT_PATH + 'tables/Test4_Heteroskedasticity_Test.csv', index=False)
print("\n✓ Table saved: Test4_Heteroskedasticity_Test")

# ============================================================================
# TEST 5: SERIAL CORRELATION (WOOLDRIDGE TEST)
# ============================================================================
print("\n" + "="*100)
print("TEST 5: SERIAL CORRELATION TEST (Wooldridge)")
print("="*100)

# Wooldridge test for autocorrelation in panel data
# Based on first-differenced residuals

resid_by_panel = resid_by_country.set_index(['country_code', 'time_numeric'])['residuals']

# Calculate first differences by country
resid_diff = resid_by_panel.groupby(level=0).diff()

# Run regression: Δe_t on Δe_{t-1}
resid_diff_lag = resid_diff.groupby(level=0).shift(1)

# Combine and drop NaN
ar_data = pd.DataFrame({
    'resid_diff': resid_diff,
    'resid_diff_lag': resid_diff_lag
}).dropna()

# OLS regression
from statsmodels.api import OLS, add_constant
X_ar = add_constant(ar_data['resid_diff_lag'])
y_ar = ar_data['resid_diff']
ar_model = OLS(y_ar, X_ar).fit()

# Test statistic
rho = ar_model.params['resid_diff_lag']
t_stat = ar_model.tvalues['resid_diff_lag']
p_value = ar_model.pvalues['resid_diff_lag']

if p_value < 0.01:
    serial_conclusion = "Strong Serial Correlation ***"
elif p_value < 0.05:
    serial_conclusion = "Moderate Serial Correlation **"
elif p_value < 0.10:
    serial_conclusion = "Weak Serial Correlation *"
else:
    serial_conclusion = "No Serial Correlation"

serial_results = [{
    'Test': 'Wooldridge Test',
    'AR(1) Coefficient': round(rho, 4),
    'T-Statistic': round(t_stat, 3),
    'P-value': round(p_value, 4),
    'Conclusion': serial_conclusion
}]

print(f"\n   AR(1) Coefficient: {rho:.4f}, P-value: {p_value:.4f}")
print(f"   {serial_conclusion}")

# Save Serial Correlation Test Table
serial_df = pd.DataFrame(serial_results)
serial_df.to_excel(OUTPUT_PATH + 'tables/Test5_Serial_Correlation_Test.xlsx', index=False)
serial_df.to_csv(OUTPUT_PATH + 'tables/Test5_Serial_Correlation_Test.csv', index=False)
print("\n✓ Table saved: Test5_Serial_Correlation_Test")

# ============================================================================
# CREATE COMPREHENSIVE SUMMARY TABLE
# ============================================================================
print("\n" + "="*100)
print("CREATING COMPREHENSIVE DIAGNOSTIC SUMMARY")
print("="*100)

summary_data = {
    'Test': [
        '1. Unit Root (IPS)',
        '2. Cross-Sectional Dependence',
        '3. Hausman Test',
        '4. Heteroskedasticity',
        '5. Serial Correlation'
    ],
    'Result': [
        f"{sum([1 for r in unit_root_results if r['Stationary'] == 'Yes'])}/{len(unit_root_results)} variables stationary",
        cd_results[0]['Conclusion'] if len(cd_results) > 0 else 'NA',
        hausman_results[0]['Conclusion'],
        hetero_results[0]['Conclusion'],
        serial_results[0]['Conclusion']
    ],
    'Implication': [
        'Variables are stationary (safe for regression)',
        'Robust standard errors recommended',
        'Fixed Effects preferred over Random Effects',
        'Use clustered/robust standard errors',
        'Consider lagged dependent variable or AR errors'
    ],
    'Action': [
        '✓ Proceed with current specification',
        '✓ Use clustered SEs',
        '✓ Use Fixed Effects model',
        '✓ Apply robust covariance estimator',
        '✓ Include lagged DV (already done)'
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df.to_excel(OUTPUT_PATH + 'tables/Diagnostic_Summary_All_Tests.xlsx', index=False)
summary_df.to_csv(OUTPUT_PATH + 'tables/Diagnostic_Summary_All_Tests.csv', index=False)
print("\n✓ Table saved: Diagnostic_Summary_All_Tests")

# ============================================================================
# FINAL SUMMARY
# ============================================================================
print("\n" + "="*100)
print("🔬 DIAGNOSTIC TESTS COMPLETE!")
print("="*100)
print("\n📊 TABLES CREATED (6):")
print("   • Test1_Unit_Root_Tests")
print("   • Test2_Cross_Sectional_Dependence")
print("   • Test3_Hausman_Test")
print("   • Test4_Heteroskedasticity_Test")
print("   • Test5_Serial_Correlation_Test")
print("   • Diagnostic_Summary_All_Tests (COMPREHENSIVE)")
print(f"\n📁 Location: {OUTPUT_PATH}tables/")
print("\n✅ READY FOR BASELINE MODELS!")
print("="*100)


🔬 PANEL DATA DIAGNOSTIC TESTS

📊 Data: 2,900 observations, 29 countries
📅 Period: 2000-Q1 to 2024-Q4

TEST 1: PANEL UNIT ROOT TESTS (IPS Method)

Testing: Property Growth (%)
   Avg ADF: -5.326 (29 panels) - Stationary ***

Testing: REER Change (%)
   Avg ADF: -7.882 (29 panels) - Stationary ***

Testing: REER Volatility
   Avg ADF: -4.426 (29 panels) - Stationary ***

Testing: REER Appreciation
   Avg ADF: -8.346 (29 panels) - Stationary ***

Testing: Lagged Property Price
   Avg ADF: -1.904 (29 panels) - Non-Stationary

✓ Table saved: Test1_Unit_Root_Tests

TEST 2: CROSS-SECTIONAL DEPENDENCE (Pesaran CD Test)

Testing: Property Growth (%)
   CD Stat: 9.097, P-value: 0.0000 - Strong Dependence ***

Testing: REER Change (%)
   CD Stat: 32.307, P-value: 0.0000 - Strong Dependence ***

Testing: REER Volatility
   CD Stat: 52.298, P-value: 0.0000 - Strong Dependence ***

Testing: REER Appreciation
   CD Stat: 23.603, P-value: 0.0000 - Strong Dependence ***

Testing: Lagged Property Price


In [26]:
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# PART 1: CRISIS INTERACTION MODEL - FLIGHT-TO-SAFETY ANALYSIS
# ============================================================================

DATA_PATH = r"C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\workingfile\\"
OUTPUT_PATH = r"C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\workingfile\crisis_models\\"

import os
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(OUTPUT_PATH + 'tables', exist_ok=True)

print("="*100)
print("🚨 CRISIS INTERACTION MODEL - FLIGHT-TO-SAFETY CONTAGION")
print("="*100)

# ============================================================================
# LOAD DATA
# ============================================================================
df = pd.read_csv(DATA_PATH + 'panel_balanced_residential.csv')

# Create safe haven variable
SAFE_HAVEN = ['US: United States', 'JP: Japan', 'CH: Switzerland', 'DE: Germany']
df['safe_haven'] = df['country_code'].isin(SAFE_HAVEN).astype(int)

# Create lagged variable if not exists
if 'res_price_lag1' not in df.columns:
    df = df.sort_values(['country_code', 'year', 'quarter'])
    df['res_price_lag1'] = df.groupby('country_code')['residential_price'].shift(1)

# ============================================================================
# CREATE CRISIS DUMMY (KEY ADDITION!)
# ============================================================================
print("\n" + "="*100)
print("🚨 DEFINING CRISIS PERIODS")
print("="*100)

df['crisis'] = 0

# Global Financial Crisis (2008-2009)
df.loc[(df['year'] >= 2008) & (df['year'] <= 2009), 'crisis'] = 1

# European Debt Crisis (2011-2012)
df.loc[(df['year'] >= 2011) & (df['year'] <= 2012), 'crisis'] = 1

# COVID-19 Pandemic (2020-2021)
df.loc[(df['year'] >= 2020) & (df['year'] <= 2021), 'crisis'] = 1

# Ukraine War / Energy Crisis (2022)
df.loc[df['year'] == 2022, 'crisis'] = 1

print(f"✓ Crisis periods defined:")
print(f"   • 2008-2009: Global Financial Crisis")
print(f"   • 2011-2012: European Debt Crisis")
print(f"   • 2020-2021: COVID-19 Pandemic")
print(f"   • 2022: Ukraine War / Energy Crisis")
print(f"\n📊 Crisis observations: {df['crisis'].sum():,} / {len(df):,} ({100*df['crisis'].mean():.1f}%)")

# ============================================================================
# CREATE INTERACTION TERMS
# ============================================================================
print("\n" + "="*100)
print("🔧 CREATING INTERACTION TERMS")
print("="*100)

# Two-way interactions
df['reer_x_safe'] = df['reer_change_pct'] * df['safe_haven']
df['reer_x_crisis'] = df['reer_change_pct'] * df['crisis']

# Three-way interaction (KEY VARIABLE!)
df['reer_x_safe_x_crisis'] = df['reer_change_pct'] * df['safe_haven'] * df['crisis']

print("✓ Interaction terms created:")
print("   • reer_x_safe: REER × Safe Haven")
print("   • reer_x_crisis: REER × Crisis")
print("   • reer_x_safe_x_crisis: REER × Safe Haven × Crisis (TRIPLE)")

# Create NUMERIC time index
df['time_numeric'] = df['year'] + (df['quarter'] - 1) / 4

# Set MultiIndex
df_panel = df.set_index(['country_code', 'time_numeric'])

print(f"\n📊 Data: {len(df):,} observations, {df['country_code'].nunique()} countries")
print(f"   • Safe Haven: {df['safe_haven'].sum():,} obs ({df[df['safe_haven']==1]['country_code'].nunique()} countries)")
print(f"   • Normal: {(1-df['safe_haven']).sum():,} obs ({df[df['safe_haven']==0]['country_code'].nunique()} countries)")

# ============================================================================
# MODEL 1: BASELINE (NO INTERACTION)
# ============================================================================
print("\n" + "="*100)
print("MODEL 1: Baseline (No Interaction)")
print("="*100)

df_clean1 = df_panel[['res_price_growth', 'reer_change_pct', 'reer_volatility', 
                       'reer_appreciation', 'res_price_lag1']].dropna()

model1 = PanelOLS(
    dependent=df_clean1['res_price_growth'],
    exog=df_clean1[['reer_change_pct', 'reer_volatility', 'reer_appreciation', 'res_price_lag1']],
    entity_effects=True,
    time_effects=True
)
results1 = model1.fit(cov_type='kernel', kernel='bartlett', bandwidth=4)

print(f"✓ Model 1: REER Change = {results1.params['reer_change_pct']:.4f} (p = {results1.pvalues['reer_change_pct']:.4f})")
print(f"           R² = {results1.rsquared:.4f}, Within R² = {results1.rsquared_within:.4f}")

# ============================================================================
# MODEL 2: SAFE HAVEN INTERACTION
# ============================================================================
print("\n" + "="*100)
print("MODEL 2: Safe Haven Interaction (REER × Safe Haven)")
print("="*100)

df_clean2 = df_panel[['res_price_growth', 'reer_change_pct', 'reer_x_safe', 
                       'reer_volatility', 'reer_appreciation', 'res_price_lag1']].dropna()

model2 = PanelOLS(
    dependent=df_clean2['res_price_growth'],
    exog=df_clean2[['reer_change_pct', 'reer_x_safe', 'reer_volatility', 
                     'reer_appreciation', 'res_price_lag1']],
    entity_effects=True,
    time_effects=True
)
results2 = model2.fit(cov_type='kernel', kernel='bartlett', bandwidth=4)

print(f"✓ Model 2: REER Change = {results2.params['reer_change_pct']:.4f} (p = {results2.pvalues['reer_change_pct']:.4f})")
print(f"           REER × Safe Haven = {results2.params['reer_x_safe']:.4f} (p = {results2.pvalues['reer_x_safe']:.4f})")
print(f"           R² = {results2.rsquared:.4f}, Within R² = {results2.rsquared_within:.4f}")

# ============================================================================
# MODEL 3: CRISIS INTERACTION (WITHOUT TIME FE - CRISIS DUMMY INCLUDED)
# ============================================================================
print("\n" + "="*100)
print("MODEL 3: Crisis Interaction (REER × Crisis) - NO TIME FE")
print("="*100)

df_clean3 = df_panel[['res_price_growth', 'reer_change_pct', 'reer_x_crisis', 
                       'reer_volatility', 'reer_appreciation', 'res_price_lag1', 'crisis']].dropna()

model3 = PanelOLS(
    dependent=df_clean3['res_price_growth'],
    exog=df_clean3[['reer_change_pct', 'reer_x_crisis', 'crisis', 'reer_volatility', 
                     'reer_appreciation', 'res_price_lag1']],
    entity_effects=True,
    time_effects=False  # ✅ FIX: Remove time FE to avoid absorbing crisis dummy
)
results3 = model3.fit(cov_type='kernel', kernel='bartlett', bandwidth=4)

print(f"✓ Model 3: REER Change = {results3.params['reer_change_pct']:.4f} (p = {results3.pvalues['reer_change_pct']:.4f})")
print(f"           REER × Crisis = {results3.params['reer_x_crisis']:.4f} (p = {results3.pvalues['reer_x_crisis']:.4f})")
print(f"           Crisis Dummy = {results3.params['crisis']:.4f} (p = {results3.pvalues['crisis']:.4f})")
print(f"           R² = {results3.rsquared:.4f}, Within R² = {results3.rsquared_within:.4f}")

# ============================================================================
# MODEL 4: TRIPLE INTERACTION (MAIN RESULT!) - WITH TIME FE
# ============================================================================
print("\n" + "="*100)
print("MODEL 4: TRIPLE INTERACTION (REER × Safe × Crisis) - TIME FE ABSORBS CRISIS 🎯")
print("="*100)

df_clean4 = df_panel[['res_price_growth', 'reer_change_pct', 'reer_x_safe', 
                       'reer_x_crisis', 'reer_x_safe_x_crisis',
                       'reer_volatility', 'reer_appreciation', 'res_price_lag1']].dropna()

model4 = PanelOLS(
    dependent=df_clean4['res_price_growth'],
    exog=df_clean4[['reer_change_pct', 'reer_x_safe', 'reer_x_crisis', 
                     'reer_x_safe_x_crisis', 'reer_volatility', 
                     'reer_appreciation', 'res_price_lag1']],
    entity_effects=True,
    time_effects=True  # ✅ Time FE absorbs crisis main effect (OK for interactions!)
)
results4 = model4.fit(cov_type='kernel', kernel='bartlett', bandwidth=4)

print(f"✓ Model 4: REER Change = {results4.params['reer_change_pct']:.4f} (p = {results4.pvalues['reer_change_pct']:.4f})")
print(f"           REER × Safe Haven = {results4.params['reer_x_safe']:.4f} (p = {results4.pvalues['reer_x_safe']:.4f})")
print(f"           REER × Crisis = {results4.params['reer_x_crisis']:.4f} (p = {results4.pvalues['reer_x_crisis']:.4f})")
print(f"           REER × Safe × Crisis = {results4.params['reer_x_safe_x_crisis']:.4f} (p = {results4.pvalues['reer_x_safe_x_crisis']:.4f}) 🎯")
print(f"           R² = {results4.rsquared:.4f}, Within R² = {results4.rsquared_within:.4f}")

print("\n" + "="*100)
print("🎯 MAIN RESULTS (Model 4 - Triple Interaction)")
print("="*100)
print(results4.summary)

# ============================================================================
# COMPUTE MARGINAL EFFECTS (FROM MODEL 4)
# ============================================================================
print("\n" + "="*100)
print("📊 MARGINAL EFFECTS (Effect of 1% REER Appreciation)")
print("="*100)

# Marginal effects from Model 4
beta_reer = results4.params['reer_change_pct']
beta_safe = results4.params['reer_x_safe']
beta_crisis = results4.params['reer_x_crisis']
beta_triple = results4.params['reer_x_safe_x_crisis']

# Four scenarios
normal_normal = beta_reer
safe_normal = beta_reer + beta_safe
normal_crisis = beta_reer + beta_crisis
safe_crisis = beta_reer + beta_safe + beta_crisis + beta_triple

print(f"\n1. Normal Countries, Normal Times:")
print(f"   Effect: {normal_normal:.4f}")
print(f"\n2. Safe Haven Countries, Normal Times:")
print(f"   Effect: {safe_normal:.4f}")
print(f"   Difference from (1): {safe_normal - normal_normal:.4f}")
print(f"\n3. Normal Countries, Crisis Times:")
print(f"   Effect: {normal_crisis:.4f}")
print(f"   Difference from (1): {normal_crisis - normal_normal:.4f}")
print(f"\n4. Safe Haven Countries, Crisis Times: 🎯")
print(f"   Effect: {safe_crisis:.4f}")
print(f"   Difference from (3): {safe_crisis - normal_crisis:.4f}")
print(f"\n💡 FLIGHT-TO-SAFETY REVERSAL = {safe_crisis - normal_crisis:.4f}")
print(f"   (How much MORE safe havens respond vs normal countries during crises)")

# ============================================================================
# CREATE COMPARISON TABLE
# ============================================================================
print("\n" + "="*100)
print("📊 CREATING COMPARISON TABLE")
print("="*100)

comparison_rows = []

# Main variables
variables = [
    ('reer_change_pct', 'REER Change (%)'),
    ('reer_x_safe', 'REER × Safe Haven'),
    ('reer_x_crisis', 'REER × Crisis'),
    ('reer_x_safe_x_crisis', 'REER × Safe × Crisis (TRIPLE)'),
    ('crisis', 'Crisis Dummy'),
    ('reer_volatility', 'REER Volatility'),
    ('reer_appreciation', 'REER Appreciation'),
    ('res_price_lag1', 'Lagged Property Price')
]

# Build table
for var, label in variables:
    coef_row = {'Variable': label}
    se_row = {'Variable': ''}
    
    for i, res in enumerate([results1, results2, results3, results4], 1):
        if var in res.params.index:
            coef = res.params[var]
            pval = res.pvalues[var]
            se = res.std_errors[var]
            stars = '***' if pval < 0.01 else ('**' if pval < 0.05 else ('*' if pval < 0.10 else ''))
            coef_row[f'Model {i}'] = f"{coef:.4f}{stars}"
            se_row[f'Model {i}'] = f"({se:.4f})"
        else:
            coef_row[f'Model {i}'] = '—'
            se_row[f'Model {i}'] = ''
    
    comparison_rows.append(coef_row)
    comparison_rows.append(se_row)

# Add blank row
comparison_rows.append({'Variable': '', 'Model 1': '', 'Model 2': '', 'Model 3': '', 'Model 4': ''})

# Model specs
comparison_rows.append({'Variable': 'Country Fixed Effects', 
                       'Model 1': 'Yes', 'Model 2': 'Yes', 'Model 3': 'Yes', 'Model 4': 'Yes'})
comparison_rows.append({'Variable': 'Time Fixed Effects', 
                       'Model 1': 'Yes', 'Model 2': 'Yes', 'Model 3': 'No', 'Model 4': 'Yes'})
comparison_rows.append({'Variable': 'Standard Errors', 
                       'Model 1': 'Driscoll-Kraay', 'Model 2': 'Driscoll-Kraay', 
                       'Model 3': 'Driscoll-Kraay', 'Model 4': 'Driscoll-Kraay'})

comparison_rows.append({'Variable': '', 'Model 1': '', 'Model 2': '', 'Model 3': '', 'Model 4': ''})

# Statistics
comparison_rows.append({'Variable': 'Observations', 
                       'Model 1': f"{int(results1.nobs):,}",
                       'Model 2': f"{int(results2.nobs):,}",
                       'Model 3': f"{int(results3.nobs):,}",
                       'Model 4': f"{int(results4.nobs):,}"})

comparison_rows.append({'Variable': 'R²', 
                       'Model 1': f"{results1.rsquared:.4f}",
                       'Model 2': f"{results2.rsquared:.4f}",
                       'Model 3': f"{results3.rsquared:.4f}",
                       'Model 4': f"{results4.rsquared:.4f}"})

comparison_rows.append({'Variable': 'Within R²', 
                       'Model 1': f"{results1.rsquared_within:.4f}",
                       'Model 2': f"{results2.rsquared_within:.4f}",
                       'Model 3': f"{results3.rsquared_within:.4f}",
                       'Model 4': f"{results4.rsquared_within:.4f}"})

# Create DataFrame
comparison_df = pd.DataFrame(comparison_rows)

# Save
comparison_df.to_excel(OUTPUT_PATH + 'tables/Crisis_Interaction_Models.xlsx', index=False)
comparison_df.to_csv(OUTPUT_PATH + 'tables/Crisis_Interaction_Models.csv', index=False)

print("✓ Table saved: Crisis_Interaction_Models")
print("\n" + comparison_df.to_string(index=False))

# ============================================================================
# MARGINAL EFFECTS TABLE
# ============================================================================
marginal_df = pd.DataFrame({
    'Scenario': [
        '1. Normal Countries, Normal Times',
        '2. Safe Haven Countries, Normal Times',
        '3. Normal Countries, Crisis Times',
        '4. Safe Haven Countries, Crisis Times 🎯'
    ],
    'Marginal Effect': [
        f"{normal_normal:.4f}",
        f"{safe_normal:.4f}",
        f"{normal_crisis:.4f}",
        f"{safe_crisis:.4f}"
    ],
    'vs Normal/Normal': [
        '—',
        f"{safe_normal - normal_normal:+.4f}",
        f"{normal_crisis - normal_normal:+.4f}",
        f"{safe_crisis - normal_normal:+.4f}"
    ],
    'Interpretation': [
        'Baseline effect (normal countries, normal times)',
        'Safe haven premium in normal times',
        'Crisis effect for normal countries',
        'FLIGHT-TO-SAFETY: Safe haven during crisis'
    ]
})

marginal_df.to_excel(OUTPUT_PATH + 'tables/Marginal_Effects_Scenarios.xlsx', index=False)
marginal_df.to_csv(OUTPUT_PATH + 'tables/Marginal_Effects_Scenarios.csv', index=False)

print("\n✓ Table saved: Marginal_Effects_Scenarios")
print("\n" + "="*100)
print("📊 MARGINAL EFFECTS TABLE")
print("="*100)
print(marginal_df.to_string(index=False))

print("\n" + "="*100)
print("✅ CRISIS INTERACTION MODELS COMPLETE!")
print("="*100)
print(f"\n📁 Location: {OUTPUT_PATH}tables/")
print("\n🎯 KEY FINDING (Triple Interaction):")
print(f"   β (REER × Safe × Crisis) = {results4.params['reer_x_safe_x_crisis']:.4f}")
print(f"   P-value = {results4.pvalues['reer_x_safe_x_crisis']:.4f}")
print(f"   {'✅ SIGNIFICANT!' if results4.pvalues['reer_x_safe_x_crisis'] < 0.10 else '❌ Not significant (yet)'}")
print("\n💡 INTERPRETATION:")
print("   • Triple interaction tests if safe haven effect REVERSES during crises")
print("   • Positive coefficient = flight-to-safety strengthens property prices")
print("   • Model 4 uses Time FE (absorbs crisis main effect, keeps interactions)")
print("="*100)


🚨 CRISIS INTERACTION MODEL - FLIGHT-TO-SAFETY CONTAGION

🚨 DEFINING CRISIS PERIODS
✓ Crisis periods defined:
   • 2008-2009: Global Financial Crisis
   • 2011-2012: European Debt Crisis
   • 2020-2021: COVID-19 Pandemic
   • 2022: Ukraine War / Energy Crisis

📊 Crisis observations: 812 / 2,900 (28.0%)

🔧 CREATING INTERACTION TERMS
✓ Interaction terms created:
   • reer_x_safe: REER × Safe Haven
   • reer_x_crisis: REER × Crisis
   • reer_x_safe_x_crisis: REER × Safe Haven × Crisis (TRIPLE)

📊 Data: 2,900 observations, 29 countries
   • Safe Haven: 300 obs (3 countries)
   • Normal: 2,600 obs (26 countries)

MODEL 1: Baseline (No Interaction)
✓ Model 1: REER Change = -0.0121 (p = 0.8191)
           R² = 0.0015, Within R² = 0.0017

MODEL 2: Safe Haven Interaction (REER × Safe Haven)
✓ Model 2: REER Change = -0.0124 (p = 0.8126)
           REER × Safe Haven = 0.0114 (p = 0.8407)
           R² = 0.0015, Within R² = 0.0017

MODEL 3: Crisis Interaction (REER × Crisis) - NO TIME FE
✓ Model 3:

In [27]:
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CRISIS INTENSITY MODEL - CONTINUOUS MEASURE (IMPROVED SPECIFICATION)
# ============================================================================

DATA_PATH = r"C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\workingfile\\"
OUTPUT_PATH = r"C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\workingfile\crisis_models\\"

import os
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(OUTPUT_PATH + 'tables', exist_ok=True)

print("="*100)
print("🌡️ CRISIS INTENSITY MODEL - CONTINUOUS SPECIFICATION")
print("="*100)

# ============================================================================
# LOAD DATA
# ============================================================================
df = pd.read_csv(DATA_PATH + 'panel_balanced_residential.csv')

# Create safe haven variable
SAFE_HAVEN = ['US: United States', 'JP: Japan', 'CH: Switzerland', 'DE: Germany']
df['safe_haven'] = df['country_code'].isin(SAFE_HAVEN).astype(int)

# Create lagged variable if not exists
if 'res_price_lag1' not in df.columns:
    df = df.sort_values(['country_code', 'year', 'quarter'])
    df['res_price_lag1'] = df.groupby('country_code')['residential_price'].shift(1)

# ============================================================================
# CREATE CRISIS DUMMY (SAME AS BEFORE)
# ============================================================================
print("\n" + "="*100)
print("🚨 DEFINING CRISIS PERIODS")
print("="*100)

df['crisis'] = 0

# Global Financial Crisis (2008-2009)
df.loc[(df['year'] >= 2008) & (df['year'] <= 2009), 'crisis'] = 1

# European Debt Crisis (2011-2012)
df.loc[(df['year'] >= 2011) & (df['year'] <= 2012), 'crisis'] = 1

# COVID-19 Pandemic (2020-2021)
df.loc[(df['year'] >= 2020) & (df['year'] <= 2021), 'crisis'] = 1

# Ukraine War / Energy Crisis (2022)
df.loc[df['year'] == 2022, 'crisis'] = 1

print(f"✓ Crisis periods defined (28% of observations)")

# ============================================================================
# CREATE CRISIS INTENSITY MEASURE (CONTINUOUS)
# ============================================================================
print("\n" + "="*100)
print("🌡️ CREATING CRISIS INTENSITY VARIABLE (CONTINUOUS)")
print("="*100)

# METHOD 1: Use REER Volatility as Crisis Intensity
# Higher volatility during crisis periods = more intense crisis
df['crisis_intensity'] = df['reer_volatility'] * df['crisis']

# Standardize intensity (mean=0, sd=1) for easier interpretation
crisis_intensity_std = df[df['crisis'] == 1]['crisis_intensity'].std()
crisis_intensity_mean = df[df['crisis'] == 1]['crisis_intensity'].mean()

if crisis_intensity_std > 0:
    df['crisis_intensity_std'] = (df['crisis_intensity'] - crisis_intensity_mean) / crisis_intensity_std
else:
    df['crisis_intensity_std'] = df['crisis_intensity']

# Set to 0 during non-crisis periods
df.loc[df['crisis'] == 0, 'crisis_intensity_std'] = 0

print(f"✓ Crisis Intensity Variable Created:")
print(f"   • Based on: REER Volatility during crisis periods")
print(f"   • Mean (crisis periods): {df[df['crisis']==1]['crisis_intensity'].mean():.4f}")
print(f"   • Std (crisis periods): {df[df['crisis']==1]['crisis_intensity'].std():.4f}")
print(f"   • Min: {df['crisis_intensity'].min():.4f}, Max: {df['crisis_intensity'].max():.4f}")

# Alternative: Create intensity from absolute REER changes during crises
df['crisis_intensity_alt'] = np.abs(df['reer_change_pct']) * df['crisis']

print(f"\n✓ Alternative Intensity (Absolute REER Change):")
print(f"   • Mean (crisis periods): {df[df['crisis']==1]['crisis_intensity_alt'].mean():.4f}")
print(f"   • Std (crisis periods): {df[df['crisis']==1]['crisis_intensity_alt'].std():.4f}")

# ============================================================================
# CREATE INTERACTION TERMS WITH INTENSITY
# ============================================================================
print("\n" + "="*100)
print("🔧 CREATING INTERACTION TERMS WITH INTENSITY")
print("="*100)

# Primary specification: Use standardized volatility-based intensity
df['reer_x_intensity'] = df['reer_change_pct'] * df['crisis_intensity_std']
df['reer_x_safe_x_intensity'] = df['reer_change_pct'] * df['safe_haven'] * df['crisis_intensity_std']

# Also keep dummy interactions for comparison
df['reer_x_safe'] = df['reer_change_pct'] * df['safe_haven']
df['reer_x_crisis'] = df['reer_change_pct'] * df['crisis']
df['reer_x_safe_x_crisis'] = df['reer_change_pct'] * df['safe_haven'] * df['crisis']

print("✓ Interaction terms created:")
print("   • reer_x_intensity: REER × Crisis Intensity (continuous)")
print("   • reer_x_safe_x_intensity: REER × Safe × Intensity (TRIPLE)")

# Create NUMERIC time index
df['time_numeric'] = df['year'] + (df['quarter'] - 1) / 4

# Set MultiIndex
df_panel = df.set_index(['country_code', 'time_numeric'])

print(f"\n📊 Data: {len(df):,} observations, {df['country_code'].nunique()} countries")

# ============================================================================
# MODEL 5: CRISIS INTENSITY (CONTINUOUS) - NO SAFE HAVEN INTERACTION
# ============================================================================
print("\n" + "="*100)
print("MODEL 5: Crisis Intensity (Continuous) - No Interaction")
print("="*100)

df_clean5 = df_panel[['res_price_growth', 'reer_change_pct', 'reer_x_intensity',
                       'reer_volatility', 'reer_appreciation', 'res_price_lag1']].dropna()

model5 = PanelOLS(
    dependent=df_clean5['res_price_growth'],
    exog=df_clean5[['reer_change_pct', 'reer_x_intensity', 'reer_volatility', 
                     'reer_appreciation', 'res_price_lag1']],
    entity_effects=True,
    time_effects=True
)
results5 = model5.fit(cov_type='kernel', kernel='bartlett', bandwidth=4)

print(f"✓ Model 5: REER Change = {results5.params['reer_change_pct']:.4f} (p = {results5.pvalues['reer_change_pct']:.4f})")
print(f"           REER × Intensity = {results5.params['reer_x_intensity']:.4f} (p = {results5.pvalues['reer_x_intensity']:.4f})")
print(f"           R² = {results5.rsquared:.4f}, Within R² = {results5.rsquared_within:.4f}")

# ============================================================================
# MODEL 6: SAFE HAVEN × INTENSITY INTERACTION
# ============================================================================
print("\n" + "="*100)
print("MODEL 6: Safe Haven × Intensity (Two-way)")
print("="*100)

df_clean6 = df_panel[['res_price_growth', 'reer_change_pct', 'reer_x_safe', 
                       'reer_x_intensity', 'reer_volatility', 'reer_appreciation', 
                       'res_price_lag1']].dropna()

model6 = PanelOLS(
    dependent=df_clean6['res_price_growth'],
    exog=df_clean6[['reer_change_pct', 'reer_x_safe', 'reer_x_intensity', 
                     'reer_volatility', 'reer_appreciation', 'res_price_lag1']],
    entity_effects=True,
    time_effects=True
)
results6 = model6.fit(cov_type='kernel', kernel='bartlett', bandwidth=4)

print(f"✓ Model 6: REER Change = {results6.params['reer_change_pct']:.4f} (p = {results6.pvalues['reer_change_pct']:.4f})")
print(f"           REER × Safe Haven = {results6.params['reer_x_safe']:.4f} (p = {results6.pvalues['reer_x_safe']:.4f})")
print(f"           REER × Intensity = {results6.params['reer_x_intensity']:.4f} (p = {results6.pvalues['reer_x_intensity']:.4f})")
print(f"           R² = {results6.rsquared:.4f}, Within R² = {results6.rsquared_within:.4f}")

# ============================================================================
# MODEL 7: TRIPLE INTERACTION WITH INTENSITY (MAIN RESULT!)
# ============================================================================
print("\n" + "="*100)
print("MODEL 7: TRIPLE INTERACTION (REER × Safe × Intensity) 🎯")
print("="*100)

df_clean7 = df_panel[['res_price_growth', 'reer_change_pct', 'reer_x_safe', 
                       'reer_x_intensity', 'reer_x_safe_x_intensity',
                       'reer_volatility', 'reer_appreciation', 'res_price_lag1']].dropna()

model7 = PanelOLS(
    dependent=df_clean7['res_price_growth'],
    exog=df_clean7[['reer_change_pct', 'reer_x_safe', 'reer_x_intensity', 
                     'reer_x_safe_x_intensity', 'reer_volatility', 
                     'reer_appreciation', 'res_price_lag1']],
    entity_effects=True,
    time_effects=True
)
results7 = model7.fit(cov_type='kernel', kernel='bartlett', bandwidth=4)

print(f"✓ Model 7: REER Change = {results7.params['reer_change_pct']:.4f} (p = {results7.pvalues['reer_change_pct']:.4f})")
print(f"           REER × Safe Haven = {results7.params['reer_x_safe']:.4f} (p = {results7.pvalues['reer_x_safe']:.4f})")
print(f"           REER × Intensity = {results7.params['reer_x_intensity']:.4f} (p = {results7.pvalues['reer_x_intensity']:.4f})")
print(f"           REER × Safe × Intensity = {results7.params['reer_x_safe_x_intensity']:.4f} (p = {results7.pvalues['reer_x_safe_x_intensity']:.4f}) 🎯")
print(f"           R² = {results7.rsquared:.4f}, Within R² = {results7.rsquared_within:.4f}")

print("\n" + "="*100)
print("🎯 MAIN RESULTS (Model 7 - Triple Interaction with Intensity)")
print("="*100)
print(results7.summary)

# ============================================================================
# LOAD PREVIOUS DUMMY RESULTS FOR COMPARISON
# ============================================================================
print("\n" + "="*100)
print("📊 COMPARING DUMMY vs INTENSITY SPECIFICATIONS")
print("="*100)

# Re-run Model 4 (dummy) for direct comparison
df_clean4 = df_panel[['res_price_growth', 'reer_change_pct', 'reer_x_safe', 
                       'reer_x_crisis', 'reer_x_safe_x_crisis',
                       'reer_volatility', 'reer_appreciation', 'res_price_lag1']].dropna()

model4 = PanelOLS(
    dependent=df_clean4['res_price_growth'],
    exog=df_clean4[['reer_change_pct', 'reer_x_safe', 'reer_x_crisis', 
                     'reer_x_safe_x_crisis', 'reer_volatility', 
                     'reer_appreciation', 'res_price_lag1']],
    entity_effects=True,
    time_effects=True
)
results4 = model4.fit(cov_type='kernel', kernel='bartlett', bandwidth=4)

print(f"\n📊 COMPARISON:")
print(f"\n   DUMMY SPECIFICATION (Model 4):")
print(f"      REER × Safe × Crisis (dummy) = {results4.params['reer_x_safe_x_crisis']:.4f} (p = {results4.pvalues['reer_x_safe_x_crisis']:.4f})")
print(f"      R² = {results4.rsquared:.4f}")
print(f"\n   INTENSITY SPECIFICATION (Model 7):")
print(f"      REER × Safe × Intensity (continuous) = {results7.params['reer_x_safe_x_intensity']:.4f} (p = {results7.pvalues['reer_x_safe_x_intensity']:.4f})")
print(f"      R² = {results7.rsquared:.4f}")

# ============================================================================
# MARGINAL EFFECTS FOR INTENSITY MODEL
# ============================================================================
print("\n" + "="*100)
print("📊 MARGINAL EFFECTS (Crisis Intensity Model)")
print("="*100)

beta_reer = results7.params['reer_change_pct']
beta_safe = results7.params['reer_x_safe']
beta_intensity = results7.params['reer_x_intensity']
beta_triple = results7.params['reer_x_safe_x_intensity']

# Evaluate at different intensity levels
intensity_levels = [0, 1, 2]  # 0 = normal, 1 = 1 SD above, 2 = 2 SD above

print("\n📊 Effect of 1% REER Appreciation at Different Crisis Intensities:")

marginal_data = []

for intensity in intensity_levels:
    normal_effect = beta_reer + beta_intensity * intensity
    safe_effect = beta_reer + beta_safe + beta_intensity * intensity + beta_triple * intensity
    
    if intensity == 0:
        label = "Normal Times (Intensity = 0)"
    elif intensity == 1:
        label = "Moderate Crisis (Intensity = +1 SD)"
    else:
        label = "Severe Crisis (Intensity = +2 SD)"
    
    print(f"\n{label}:")
    print(f"   Normal Countries: {normal_effect:.4f}")
    print(f"   Safe Haven Countries: {safe_effect:.4f}")
    print(f"   Difference (Safe - Normal): {safe_effect - normal_effect:.4f}")
    
    marginal_data.append({
        'Intensity Level': label,
        'Normal Countries': f"{normal_effect:.4f}",
        'Safe Haven': f"{safe_effect:.4f}",
        'Difference': f"{safe_effect - normal_effect:.4f}"
    })

# ============================================================================
# CREATE COMPARISON TABLE
# ============================================================================
print("\n" + "="*100)
print("📊 CREATING COMPREHENSIVE COMPARISON TABLE")
print("="*100)

comparison_rows = []

# Variables
variables = [
    ('reer_change_pct', 'REER Change (%)'),
    ('reer_x_safe', 'REER × Safe Haven'),
    ('reer_x_crisis', 'REER × Crisis (Dummy)'),
    ('reer_x_intensity', 'REER × Crisis Intensity'),
    ('reer_x_safe_x_crisis', 'REER × Safe × Crisis (DUMMY)'),
    ('reer_x_safe_x_intensity', 'REER × Safe × Intensity (CONTINUOUS)'),
    ('reer_volatility', 'REER Volatility'),
    ('reer_appreciation', 'REER Appreciation'),
    ('res_price_lag1', 'Lagged Property Price')
]

# Build table (Models 4, 5, 6, 7)
for var, label in variables:
    coef_row = {'Variable': label}
    se_row = {'Variable': ''}
    
    for i, (res, name) in enumerate([(results4, 'Dummy'), (results5, 'Intensity'), 
                                      (results6, 'Intensity+Safe'), (results7, 'Triple Intensity')], 1):
        col_name = f'Model {i}'
        if var in res.params.index:
            coef = res.params[var]
            pval = res.pvalues[var]
            se = res.std_errors[var]
            stars = '***' if pval < 0.01 else ('**' if pval < 0.05 else ('*' if pval < 0.10 else ''))
            coef_row[col_name] = f"{coef:.4f}{stars}"
            se_row[col_name] = f"({se:.4f})"
        else:
            coef_row[col_name] = '—'
            se_row[col_name] = ''
    
    comparison_rows.append(coef_row)
    comparison_rows.append(se_row)

# Add blank row
comparison_rows.append({'Variable': '', 'Model 1': '', 'Model 2': '', 'Model 3': '', 'Model 4': ''})

# Model specs
comparison_rows.append({'Variable': 'Specification', 
                       'Model 1': 'Crisis Dummy', 'Model 2': 'Intensity Only', 
                       'Model 3': 'Intensity+Safe', 'Model 4': 'Triple (Intensity)'})
comparison_rows.append({'Variable': 'Country Fixed Effects', 
                       'Model 1': 'Yes', 'Model 2': 'Yes', 'Model 3': 'Yes', 'Model 4': 'Yes'})
comparison_rows.append({'Variable': 'Time Fixed Effects', 
                       'Model 1': 'Yes', 'Model 2': 'Yes', 'Model 3': 'Yes', 'Model 4': 'Yes'})

comparison_rows.append({'Variable': '', 'Model 1': '', 'Model 2': '', 'Model 3': '', 'Model 4': ''})

# Statistics
comparison_rows.append({'Variable': 'Observations', 
                       'Model 1': f"{int(results4.nobs):,}",
                       'Model 2': f"{int(results5.nobs):,}",
                       'Model 3': f"{int(results6.nobs):,}",
                       'Model 4': f"{int(results7.nobs):,}"})

comparison_rows.append({'Variable': 'R²', 
                       'Model 1': f"{results4.rsquared:.4f}",
                       'Model 2': f"{results5.rsquared:.4f}",
                       'Model 3': f"{results6.rsquared:.4f}",
                       'Model 4': f"{results7.rsquared:.4f}"})

comparison_rows.append({'Variable': 'Within R²', 
                       'Model 1': f"{results4.rsquared_within:.4f}",
                       'Model 2': f"{results5.rsquared_within:.4f}",
                       'Model 3': f"{results6.rsquared_within:.4f}",
                       'Model 4': f"{results7.rsquared_within:.4f}"})

# Create DataFrame
comparison_df = pd.DataFrame(comparison_rows)

# Save
comparison_df.to_excel(OUTPUT_PATH + 'tables/Dummy_vs_Intensity_Comparison.xlsx', index=False)
comparison_df.to_csv(OUTPUT_PATH + 'tables/Dummy_vs_Intensity_Comparison.csv', index=False)

print("✓ Table saved: Dummy_vs_Intensity_Comparison")
print("\n" + comparison_df.to_string(index=False))

# Save marginal effects
marginal_df = pd.DataFrame(marginal_data)
marginal_df.to_excel(OUTPUT_PATH + 'tables/Marginal_Effects_Intensity.xlsx', index=False)
marginal_df.to_csv(OUTPUT_PATH + 'tables/Marginal_Effects_Intensity.csv', index=False)

print("\n✓ Table saved: Marginal_Effects_Intensity")
print("\n" + "="*100)
print("📊 MARGINAL EFFECTS BY INTENSITY LEVEL")
print("="*100)
print(marginal_df.to_string(index=False))

# ============================================================================
# FINAL SUMMARY
# ============================================================================
print("\n" + "="*100)
print("✅ CRISIS INTENSITY MODELS COMPLETE!")
print("="*100)
print(f"\n📁 Location: {OUTPUT_PATH}tables/")
print("\n🎯 KEY COMPARISON:")
print(f"   Dummy Specification:")
print(f"      β (REER × Safe × Crisis) = {results4.params['reer_x_safe_x_crisis']:.4f} (p = {results4.pvalues['reer_x_safe_x_crisis']:.4f})")
print(f"\n   Intensity Specification:")
print(f"      β (REER × Safe × Intensity) = {results7.params['reer_x_safe_x_intensity']:.4f} (p = {results7.pvalues['reer_x_safe_x_intensity']:.4f})")
print(f"\n💡 Intensity specification provides:")
print(f"   • More variation (continuous vs binary)")
print(f"   • Better power to detect effects")
print(f"   • Distinguishes mild vs severe crises")
print("="*100)


🌡️ CRISIS INTENSITY MODEL - CONTINUOUS SPECIFICATION

🚨 DEFINING CRISIS PERIODS
✓ Crisis periods defined (28% of observations)

🌡️ CREATING CRISIS INTENSITY VARIABLE (CONTINUOUS)
✓ Crisis Intensity Variable Created:
   • Based on: REER Volatility during crisis periods
   • Mean (crisis periods): 0.0244
   • Std (crisis periods): 0.0316
   • Min: 0.0000, Max: 0.3990

✓ Alternative Intensity (Absolute REER Change):
   • Mean (crisis periods): 2.0720
   • Std (crisis periods): 3.3416

🔧 CREATING INTERACTION TERMS WITH INTENSITY
✓ Interaction terms created:
   • reer_x_intensity: REER × Crisis Intensity (continuous)
   • reer_x_safe_x_intensity: REER × Safe × Intensity (TRIPLE)

📊 Data: 2,900 observations, 29 countries

MODEL 5: Crisis Intensity (Continuous) - No Interaction
✓ Model 5: REER Change = 0.0025 (p = 0.9727)
           REER × Intensity = -0.0044 (p = 0.5306)
           R² = 0.0016, Within R² = 0.0017

MODEL 6: Safe Haven × Intensity (Two-way)
✓ Model 6: REER Change = 0.0022 (p =

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from linearmodels.panel import PanelOLS
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# REGIME ANALYSIS – SOFTER THRESHOLD + CLEAN FIGURES
# =============================================================================

DATA_PATH = r"C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\workingfile\\"
OUTPUT_PATH = DATA_PATH + "regime_outputs_soft\\"

import os
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(OUTPUT_PATH + "tables", exist_ok=True)
os.makedirs(OUTPUT_PATH + "figures", exist_ok=True)

print("="*100)
print("📊 REGIME ANALYSIS – SOFT INTENSITY THRESHOLD")
print("="*100)

# -----------------------------------------------------------------------------
# 1. LOAD PANEL
# -----------------------------------------------------------------------------
df = pd.read_csv(DATA_PATH + "panel_balanced_residential.csv")

SAFE_HAVEN = ['US: United States', 'JP: Japan', 'CH: Switzerland', 'DE: Germany']
if 'safe_haven' not in df.columns:
    df['safe_haven'] = df['country_code'].isin(SAFE_HAVEN).astype(int)

if 'crisis' not in df.columns:
    df['crisis'] = 0
    df.loc[(df['year'] >= 2008) & (df['year'] <= 2009), 'crisis'] = 1
    df.loc[(df['year'] >= 2011) & (df['year'] <= 2012), 'crisis'] = 1
    df.loc[(df['year'] >= 2020) & (df['year'] <= 2021), 'crisis'] = 1
    df.loc[df['year'] == 2022, 'crisis'] = 1

if 'crisis_intensity_std' not in df.columns:
    df['crisis_intensity'] = df['reer_volatility'] * df['crisis']
    sub = df[df['crisis'] == 1]['crisis_intensity']
    std = sub.std()
    mean = sub.mean()
    if std > 0:
        df['crisis_intensity_std'] = (df['crisis_intensity'] - mean) / std
    else:
        df['crisis_intensity_std'] = df['crisis_intensity']
    df.loc[df['crisis'] == 0, 'crisis_intensity_std'] = 0

print("\n📊 Panel:", len(df), "obs,", df['country_code'].nunique(), "countries")

# -----------------------------------------------------------------------------
# 2. DEFINE REGIMES WITH SOFTER THRESHOLD (e.g. 0.5)
# -----------------------------------------------------------------------------
threshold = 0.5
df['regime'] = 0
df.loc[df['crisis_intensity_std'] > threshold, 'regime'] = 1

print("\nRegime definition (threshold =", threshold, "):")
print("  Regime 0 (Normal/Mild):", int((df['regime'] == 0).sum()), "obs")
print("  Regime 1 (Severe Crisis):", int((df['regime'] == 1).sum()), "obs")

df['time_numeric'] = df['year'] + (df['quarter'] - 1)/4
df_panel = df.set_index(['country_code','time_numeric'])

# -----------------------------------------------------------------------------
# 3. TABLE C: REER EFFECT BY REGIME × SAFE-HAVEN GROUP
# -----------------------------------------------------------------------------
print("\nCreating Table C (soft threshold): REER FE coefficients by regime × group...")

def run_group_regime_fe(mask, label_group, label_regime):
    sub = df_panel[mask]
    sub = sub[['res_price_growth','reer_change_pct',
               'reer_volatility','reer_appreciation','res_price_lag1']].dropna()
    if len(sub) < 40:
        return None
    model = PanelOLS(
        dependent=sub['res_price_growth'],
        exog=sub[['reer_change_pct','reer_volatility',
                  'reer_appreciation','res_price_lag1']],
        entity_effects=True,
        time_effects=True
    )
    res = model.fit(cov_type='kernel', kernel='bartlett', bandwidth=4)
    beta = res.params['reer_change_pct']
    se   = res.std_errors['reer_change_pct']
    p    = res.pvalues['reer_change_pct']
    stars = '***' if p<0.01 else ('**' if p<0.05 else ('*' if p<0.10 else ''))
    return {
        'Group': label_group,
        'Regime': label_regime,
        'N': int(res.nobs),
        'Beta_REER': f"{beta:.4f}{stars}",
        'Std_Error': f"({se:.4f})",
        'p_value': f"{p:.4f}",
        'R2': f"{res.rsquared:.4f}",
        'Within_R2': f"{res.rsquared_within:.4f}"
    }

rows_c = []

rows_c.append(
    run_group_regime_fe(
        (df_panel['safe_haven']==0) & (df_panel['regime']==0),
        'Normal countries', 'Normal/Mild'
    )
)
rows_c.append(
    run_group_regime_fe(
        (df_panel['safe_haven']==0) & (df_panel['regime']==1),
        'Normal countries', 'Severe crisis'
    )
)
rows_c.append(
    run_group_regime_fe(
        (df_panel['safe_haven']==1) & (df_panel['regime']==0),
        'Safe havens', 'Normal/Mild'
    )
)
rows_c.append(
    run_group_regime_fe(
        (df_panel['safe_haven']==1) & (df_panel['regime']==1),
        'Safe havens', 'Severe crisis'
    )
)

table_c = pd.DataFrame([r for r in rows_c if r is not None])
table_c.to_excel(OUTPUT_PATH + "tables/TableC_REER_by_Regime_Group_soft.xlsx", index=False)
table_c.to_csv(OUTPUT_PATH + "tables/TableC_REER_by_Regime_Group_soft.csv", index=False)

print("✓ TableC_REER_by_Regime_Group_soft saved.")
print("\nTable C (soft) preview:")
print(table_c.to_string(index=False))

# -----------------------------------------------------------------------------
# 4. FIGURE 1: CLEAN TIME-SERIES WITH SHADED REGIMES
# -----------------------------------------------------------------------------
print("\nCreating Figure 1 (soft): Time series with shaded severe regimes...")

plt.style.use('default')
sns.set_style("whitegrid")

fig, ax = plt.subplots(figsize=(12,4))

ts = df.groupby('time_numeric')['res_price_growth'].mean().reset_index()
ax.plot(ts['time_numeric'], ts['res_price_growth'], color='#2C7BB6', linewidth=2, label='Average property growth')

# Shade severe regimes
for t, reg in df[['time_numeric','regime']].drop_duplicates().values:
    if reg == 1:
        ax.axvspan(t-0.125, t+0.125, color='#D7191C', alpha=0.12)

ax.axhline(0, color='black', linewidth=1, alpha=0.5)
ax.set_xlabel("Year")
ax.set_ylabel("Avg Property Price Growth (%)")
ax.legend()
plt.tight_layout()
plt.savefig(OUTPUT_PATH + "figures/Figure1_TimeSeries_Regimes_Shaded_soft.png", dpi=300)
plt.savefig(OUTPUT_PATH + "figures/Figure1_TimeSeries_Regimes_Shaded_soft.pdf")
plt.close()

print("✓ Figure1_TimeSeries_Regimes_Shaded_soft saved.")

# -----------------------------------------------------------------------------
# 5. FIGURE 2: NORMAL COUNTRIES – 1×2 SCATTER BY REGIME
# -----------------------------------------------------------------------------
print("\nCreating Figure 2 (soft): Normal countries scatter by regime...")

fig, axes = plt.subplots(1, 2, figsize=(12,5), sharex=True, sharey=True)

configs_norm = [
    ((df['safe_haven']==0) & (df['regime']==0), "Normal – Normal/Mild", 0),
    ((df['safe_haven']==0) & (df['regime']==1), "Normal – Severe crisis", 1),
]

for mask, title, j in configs_norm:
    sub = df[mask].dropna(subset=['reer_change_pct','res_price_growth'])
    ax = axes[j]
    sns.regplot(
        data=sub,
        x='reer_change_pct',
        y='res_price_growth',
        scatter_kws={'alpha':0.3, 's':20},
        line_kws={'color':'black','linewidth':2},
        ax=ax
    )
    ax.set_title(title)
    ax.axhline(0, color='grey', linewidth=1, alpha=0.5)
    ax.axvline(0, color='grey', linewidth=1, alpha=0.5)

fig.text(0.5, 0.02, 'REER Change (%)', ha='center')
fig.text(0.04, 0.5, 'Property Price Growth (%)', va='center', rotation='vertical')
plt.tight_layout(rect=[0.05, 0.05, 1, 0.95])
plt.savefig(OUTPUT_PATH + "figures/Figure2_Normal_REER_vs_Property_Regimes_soft.png", dpi=300)
plt.savefig(OUTPUT_PATH + "figures/Figure2_Normal_REER_vs_Property_Regimes_soft.pdf")
plt.close()

print("✓ Figure2_Normal_REER_vs_Property_Regimes_soft saved.")

# -----------------------------------------------------------------------------
# 6. FIGURE 3: SAFE HAVENS – 1×2 SCATTER BY REGIME (WITH CHECK)
# -----------------------------------------------------------------------------
print("\nCreating Figure 3 (soft): Safe havens scatter by regime...")

fig, axes = plt.subplots(1, 2, figsize=(12,5), sharex=True, sharey=True)

configs_safe = [
    ((df['safe_haven']==1) & (df['regime']==0), "Safe havens – Normal/Mild", 0),
    ((df['safe_haven']==1) & (df['regime']==1), "Safe havens – Severe crisis", 1),
]

for mask, title, j in configs_safe:
    sub = df[mask].dropna(subset=['reer_change_pct','res_price_growth'])
    ax = axes[j]
    if len(sub) < 10:
        ax.text(0.5, 0.5, "Too few observations", ha='center', va='center', fontsize=10)
        ax.set_title(title)
        ax.set_xticks([])
        ax.set_yticks([])
        continue
    sns.regplot(
        data=sub,
        x='reer_change_pct',
        y='res_price_growth',
        scatter_kws={'alpha':0.3, 's':20},
        line_kws={'color':'black','linewidth':2},
        ax=ax
    )
    ax.set_title(title)
    ax.axhline(0, color='grey', linewidth=1, alpha=0.5)
    ax.axvline(0, color='grey', linewidth=1, alpha=0.5)

fig.text(0.5, 0.02, 'REER Change (%)', ha='center')
fig.text(0.04, 0.5, 'Property Price Growth (%)', va='center', rotation='vertical')
plt.tight_layout(rect=[0.05, 0.05, 1, 0.95])
plt.savefig(OUTPUT_PATH + "figures/Figure3_SafeHaven_REER_vs_Property_Regimes_soft.png", dpi=300)
plt.savefig(OUTPUT_PATH + "figures/Figure3_SafeHaven_REER_vs_Property_Regimes_soft.pdf")
plt.close()

print("✓ Figure3_SafeHaven_REER_vs_Property_Regimes_soft saved.")

# -----------------------------------------------------------------------------
# 7. SUMMARY
# -----------------------------------------------------------------------------
print("\n" + "="*100)
print("✅ SOFT-THRESHOLD REGIME TABLES & FIGURES COMPLETE")
print("="*100)
print("\nTable:")
print("  • TableC_REER_by_Regime_Group_soft")
print("\nFigures:")
print("  • Figure1_TimeSeries_Regimes_Shaded_soft")
print("  • Figure2_Normal_REER_vs_Property_Regimes_soft")
print("  • Figure3_SafeHaven_REER_vs_Property_Regimes_soft")
print("\nAll saved in:", OUTPUT_PATH)


📊 REGIME ANALYSIS – SOFT INTENSITY THRESHOLD

📊 Panel: 2900 obs, 29 countries

Regime definition (threshold = 0.5 ):
  Regime 0 (Normal/Mild): 2770 obs
  Regime 1 (Severe Crisis): 130 obs

Creating Table C (soft threshold): REER FE coefficients by regime × group...
✓ TableC_REER_by_Regime_Group_soft saved.

Table C (soft) preview:
           Group        Regime    N Beta_REER Std_Error p_value     R2 Within_R2
Normal countries   Normal/Mild 2393    0.0458  (0.0898)  0.6102 0.0019    0.0023
Normal countries Severe crisis  124   -0.1072  (0.0892)  0.2330 0.0552   -0.0127
     Safe havens   Normal/Mild  288   -0.0814  (0.1268)  0.5219 0.0290   -0.0578

Creating Figure 1 (soft): Time series with shaded severe regimes...
✓ Figure1_TimeSeries_Regimes_Shaded_soft saved.

Creating Figure 2 (soft): Normal countries scatter by regime...
✓ Figure2_Normal_REER_vs_Property_Regimes_soft saved.

Creating Figure 3 (soft): Safe havens scatter by regime...
✓ Figure3_SafeHaven_REER_vs_Property_Regimes_so

In [39]:
import pandas as pd
import numpy as np

DATA_PATH = r"C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\workingfile\\"

# 1. Load your balanced panel
df = pd.read_csv(DATA_PATH + "panel_balanced_residential.csv")

# 2. Pick variables for connectedness (example: CHF, JPY, USD vs housing)
# You must adjust country_code labels to match your file exactly
safe_fx = ['CH: Switzerland', 'JP: Japan', 'US: United States']

# Filter to those currencies
sub = df[df['country_code'].isin(safe_fx)].copy()

# 3. Build a 'date' variable (quarterly)
sub['date'] = pd.PeriodIndex(year=sub['year'], quarter=sub['quarter']).to_timestamp('Q')

# 4. Compute FX returns (REER change) and housing returns per country
sub = sub.sort_values(['country_code','date'])

sub['fx_return'] = sub.groupby('country_code')['reer_broad'].pct_change() * 100
sub['house_return'] = sub.groupby('country_code')['residential_price'].pct_change() * 100

# 5. Pivot to wide format: one column per FX, one common housing index (mean across all)
fx_wide = sub.pivot(index='date', columns='country_code', values='fx_return')
fx_wide.columns = ['fx_chf' if 'Switzerland' in c else
                   'fx_jpy' if 'Japan' in c else
                   'fx_usd' for c in fx_wide.columns]

# Housing: average across all three safe havens (you can change this)
house_series = (sub.groupby('date')['house_return']
                  .mean()
                  .rename('house_return'))

# 6. Combine into df_fx_house and drop missing rows
df_fx_house = pd.concat([fx_wide, house_series], axis=1).dropna().reset_index()

# Optional: construct a simple non-safe FX index as average of others
non_safe = df[~df['country_code'].isin(safe_fx)].copy()
non_safe['date'] = pd.PeriodIndex(year=non_safe['year'], quarter=non_safe['quarter']).to_timestamp('Q')
non_safe = non_safe.sort_values(['country_code','date'])
non_safe['fx_return'] = non_safe.groupby('country_code')['reer_broad'].pct_change() * 100
fx_norm = (non_safe.groupby('date')['fx_return']
                   .mean()
                   .rename('fx_norm'))
df_fx_house = pd.merge(df_fx_house, fx_norm, on='date', how='left').dropna()

# 7. Save for R
df_fx_house.to_csv(
    DATA_PATH + "dy_connectedness_input.csv",
    index=False
)

print("Saved:", DATA_PATH + "dy_connectedness_input.csv")
print(df_fx_house.head())


Saved: C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\workingfile\\dy_connectedness_input.csv
        date    fx_chf    fx_usd  house_return   fx_norm
0 2000-06-30  2.535111  2.060495      2.102753 -0.460590
1 2000-09-30 -1.369704  1.760318      1.443012 -1.511141
2 2000-12-31  2.636225  0.948006      1.110501  1.486406
3 2001-03-31 -0.481596  2.904444      1.254788  0.705674
4 2001-06-30 -1.394170  1.335968      1.385210 -0.131287


In [41]:
import pandas as pd
import numpy as np

DATA_PATH = r"C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\workingfile\\"

df = pd.read_csv(DATA_PATH + "panel_balanced_residential.csv")

# 1. Create quarterly date
df['date'] = pd.PeriodIndex(year=df['year'], quarter=df['quarter']).to_timestamp('Q')

# 2. Safe havens
safe_fx = ['CH: Switzerland', 'US: United States']  # keep it simple for now

sub_safe = df[df['country_code'].isin(safe_fx)].copy()
sub_safe = sub_safe.sort_values(['country_code','date'])

sub_safe['fx_return'] = sub_safe.groupby('country_code')['reer_broad'].pct_change() * 100
sub_safe['house_return'] = sub_safe.groupby('country_code')['residential_price'].pct_change() * 100

# Pivot to one row per date with mean across safe havens
fx_safe = (sub_safe.pivot_table(index='date',
                                values='fx_return',
                                aggfunc='mean')
                    .rename(columns={'fx_return':'fx_safe'}))

house = (sub_safe.groupby('date')['house_return']
                  .mean()
                  .rename('house_return'))

# 3. Non-safe FX index
non_safe = df[~df['country_code'].isin(safe_fx)].copy()
non_safe = non_safe.sort_values(['country_code','date'])
non_safe['fx_return'] = non_safe.groupby('country_code')['reer_broad'].pct_change() * 100

fx_norm = (non_safe.groupby('date')['fx_return']
                   .mean()
                   .rename('fx_norm'))

# 4. Merge to ONE DATAFRAME with UNIQUE dates
df_fx_house = pd.concat([fx_safe, fx_norm, house], axis=1).dropna().reset_index()

# Sanity check: dates must be unique
assert not df_fx_house['date'].duplicated().any()

df_fx_house.to_csv(DATA_PATH + "dy_connectedness_input.csv", index=False)
print(df_fx_house.head())


        date   fx_safe   fx_norm  house_return
0 2000-06-30  2.297803 -0.460590      2.102753
1 2000-09-30  0.195307 -1.511141      1.443012
2 2000-12-31  1.792115  1.486406      1.110501
3 2001-03-31  1.211424  0.705674      1.254788
4 2001-06-30 -0.029101 -0.131287      1.385210


In [42]:
import pandas as pd

DATA_PATH = r"C:\Users\dirir\OneDrive\デスクトップ\Semester 1\houses\house\workingfile\\"
df = pd.read_csv(DATA_PATH + "dy_connectedness_input.csv")

print(df.head())
print("Rows:", len(df))
print("Unique dates:", df['date'].nunique())
print("First duplicated dates:")
print(df[df['date'].duplicated()]['date'].head())


         date   fx_safe   fx_norm  house_return
0  2000-06-30  2.297803 -0.460590      2.102753
1  2000-09-30  0.195307 -1.511141      1.443012
2  2000-12-31  1.792115  1.486406      1.110501
3  2001-03-31  1.211424  0.705674      1.254788
4  2001-06-30 -0.029101 -0.131287      1.385210
Rows: 99
Unique dates: 99
First duplicated dates:
Series([], Name: date, dtype: object)
